### 필요한 라이브러리 설치

In [ ]:
!pip install pefile
!pip install numpy
!pip install sklearn
!pip install lightgbm
!pip install tqdm
!pip install pandas
!pip install graphviz
!pip install overload
!pip install lief

### 구글 드라이브 연동 및 데이터 다운로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/My\ Drive/데이터.zip

### 사용할 헤더

In [ ]:
## Tutorial
import os
import glob
import json
import pprint

import numpy as np

from lightgbm import LGBMClassifier

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE

## PE-Miner
import pickle
# import  peminer
import seaborn as sns
# from overload import overload

## Ember
# import  ember
#import pandas as pd
import  pandas as  pd
from  tqdm.notebook import  tqdm
from  sklearn import  tree
from  sklearn.metrics import  precision_score
from  sklearn.metrics import  recall_score
from  sklearn.metrics import  f1_score
from  sklearn.metrics import  plot_confusion_matrix
from  sklearn.metrics import  confusion_matrix
from  sklearn.model_selection import  GridSearchCV
import  matplotlib.pyplot as  plt
from sklearn.feature_extraction import FeatureHasher

# from  pprint import  pprint

%matplotlib inline

### 파일 읽기

In [ ]:
def read_label_csv(path):
    label_table = dict()
    with open(path, "r", encoding = "cp949") as f: #csv파일을 읽는다
        for line in f.readlines()[1:]:
            fname, label = line.strip().split(",")
            label_table[fname] = int(label)
    return label_table

def read_json(path):
    with open(path, "r") as f:  #json파일을 읽는다 
        return json.load(f)

In [ ]:
learn_label_table = read_label_csv("학습데이터_정답.csv")
verify_label_table = read_label_csv("검증데이터_정답.csv")

###특징 벡터 생성 예시

*   PEMINER 정보는 모두 수치형 데이터이므로 특별히 가공을 하지 않고 사용 가능
*   EMBER, PESTUDIO 정보는 가공해서 사용해야 할 특징들이 있음 (e.g. imports, exports 등의 문자열 정보를 가지는 데이터)
*   수치형 데이터가 아닌 데이터(범주형 데이터)를 어떻게 가공할 지가 관건 >> 인코딩 (e.g. 원핫인코딩, 레이블인코딩 등)


> #### PEMINER



In [ ]:
## 중요도가 높은 상위 10개 단위로 가져온 리스트들
# peminer_header_10 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size']
# peminer_header_20 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData']
# peminer_header_30 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size']
peminer_header_40 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress']
# peminer_header_50 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics']
# peminer_header_60 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size']
# peminer_header_70 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL']
# peminer_header_80 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL']
# peminer_header_90 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers']
# peminer_header_100 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers', 'OptionalHeader.LoaderFlags', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.Magic', 'Section.text.PointerToRelocations', 'WINSOCK.DLL', 'Section.text.NumberOfLinenumbers', 'Section.rsrc.PointerToRelocations', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'Section.rsrc.PointerToLinenumbers']
## 중요도가 0인 것을 제외한 리스트(107)
# peminer_header_107 = ['COMCTL32.DLL', 'COMDLG32.DLL', 'CRTDLL.DLL', 'DCIMAN32.DLL', 'FileHeader.Characteristics', 'GDI32.DLL', 'LZ32.DLL', 'MAPI32.DLL', 'MPR.DLL', 'NETAPI32.DLL', 'OLEDLG.DLL', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.BaseOfCode', 'OptionalHeader.BaseOfData', 'OptionalHeader.CheckSum', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'OptionalHeader.FileAlignment', 'OptionalHeader.ImageBase', 'OptionalHeader.LoaderFlags', 'OptionalHeader.Magic', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.MajorLinkerVersion', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.MinorLinkerVersion', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'OptionalHeader.SectionAlignment', 'OptionalHeader.SizeOfCode', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.SizeOfHeapReserve', 'OptionalHeader.SizeOfImage', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.SizeOfStackReserve', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.Subsystem', 'RASAPI32.DLL', 'RPCRT4.DLL', 'SECUR32.DLL', 'SHELL32.DLL', 'SHLWAPI.DLL', 'Section.data.Characteristics', 'Section.data.NumberOfLinenumbers', 'Section.data.PointerToLinenumbers', 'Section.data.PointerToRawData', 'Section.data.PointerToRelocations', 'Section.data.SizeOfRawData', 'Section.data.VirtualAddress', 'Section.data.VirtualSize', 'Section.rsrc.Characteristics', 'Section.rsrc.NumberOfLinenumbers', 'Section.rsrc.NumberOfRelocations', 'Section.rsrc.PointerToLinenumbers', 'Section.rsrc.PointerToRawData', 'Section.rsrc.PointerToRelocations', 'Section.rsrc.SizeOfRawData', 'Section.rsrc.VirtualAddress', 'Section.rsrc.VirtualSize', 'Section.text.Characteristics', 'Section.text.NumberOfLinenumbers', 'Section.text.NumberOfRelocations', 'Section.text.PointerToLinenumbers', 'Section.text.PointerToRawData', 'Section.text.PointerToRelocations', 'Section.text.SizeOfRawData', 'Section.text.VirtualAddress', 'Section.text.VirtualSize', 'USER32.DLL', 'VERSION.DLL', 'WININET.DLL', 'WINMM.DLL', 'WINSOCK.DLL', 'WSOCK32.DLL']
# peminer_header_110 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers', 'OptionalHeader.LoaderFlags', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.Magic', 'Section.text.PointerToRelocations', 'WINSOCK.DLL', 'Section.text.NumberOfLinenumbers', 'Section.rsrc.PointerToRelocations', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'Section.rsrc.PointerToLinenumbers', 'Section.text.NumberOfRelocations', 'Section.rsrc.NumberOfRelocations', 'Section.data.PointerToLinenumbers', 'MAPI32.DLL', 'Section.data.PointerToRelocations', 'Section.data.NumberOfLinenumbers', 'DCIMAN32.DLL', 'ADVAP132.DLL', 'AWFAXP32.DLL', 'AWFXAB32.DLL']
# peminer_header_120 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers', 'OptionalHeader.LoaderFlags', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.Magic', 'Section.text.PointerToRelocations', 'WINSOCK.DLL', 'Section.text.NumberOfLinenumbers', 'Section.rsrc.PointerToRelocations', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'Section.rsrc.PointerToLinenumbers', 'Section.text.NumberOfRelocations', 'Section.rsrc.NumberOfRelocations', 'Section.data.PointerToLinenumbers', 'MAPI32.DLL', 'Section.data.PointerToRelocations', 'Section.data.NumberOfLinenumbers', 'DCIMAN32.DLL', 'ADVAP132.DLL', 'AWFAXP32.DLL', 'AWFXAB32.DLL', 'AWPWD32.DLL', 'AWRESX32.DLL', 'AWUTIL32.DLL', 'BHNETB.DLL', 'BHSUPP.DLL', 'CCAPI.DLL', 'CCEI.DLL', 'CCPSH.DLL', 'CCTN20.DLL', 'CMC.DLL']
# peminer_header_130 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers', 'OptionalHeader.LoaderFlags', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.Magic', 'Section.text.PointerToRelocations', 'WINSOCK.DLL', 'Section.text.NumberOfLinenumbers', 'Section.rsrc.PointerToRelocations', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'Section.rsrc.PointerToLinenumbers', 'Section.text.NumberOfRelocations', 'Section.rsrc.NumberOfRelocations', 'Section.data.PointerToLinenumbers', 'MAPI32.DLL', 'Section.data.PointerToRelocations', 'Section.data.NumberOfLinenumbers', 'DCIMAN32.DLL', 'ADVAP132.DLL', 'AWFAXP32.DLL', 'AWFXAB32.DLL', 'AWPWD32.DLL', 'AWRESX32.DLL', 'AWUTIL32.DLL', 'BHNETB.DLL', 'BHSUPP.DLL', 'CCAPI.DLL', 'CCEI.DLL', 'CCPSH.DLL', 'CCTN20.DLL', 'CMC.DLL', 'DCIMAN.DLL', 'DSKMAINT.DLL', 'FileHeader.Machine', 'FileHeader.NumberOfSections', 'FileHeader.NumberOfSymbols', 'FileHeader.PointerToSymbolTable', 'FileHeader.SizeOfOptionalHeader', 'FileHeader.TimeDateStamp', 'GROUP.DLL', 'HYPERTERM.DLL']
# peminer_header_140 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers', 'OptionalHeader.LoaderFlags', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.Magic', 'Section.text.PointerToRelocations', 'WINSOCK.DLL', 'Section.text.NumberOfLinenumbers', 'Section.rsrc.PointerToRelocations', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'Section.rsrc.PointerToLinenumbers', 'Section.text.NumberOfRelocations', 'Section.rsrc.NumberOfRelocations', 'Section.data.PointerToLinenumbers', 'MAPI32.DLL', 'Section.data.PointerToRelocations', 'Section.data.NumberOfLinenumbers', 'DCIMAN32.DLL', 'ADVAP132.DLL', 'AWFAXP32.DLL', 'AWFXAB32.DLL', 'AWPWD32.DLL', 'AWRESX32.DLL', 'AWUTIL32.DLL', 'BHNETB.DLL', 'BHSUPP.DLL', 'CCAPI.DLL', 'CCEI.DLL', 'CCPSH.DLL', 'CCTN20.DLL', 'CMC.DLL', 'DCIMAN.DLL', 'DSKMAINT.DLL', 'FileHeader.Machine', 'FileHeader.NumberOfSections', 'FileHeader.NumberOfSymbols', 'FileHeader.PointerToSymbolTable', 'FileHeader.SizeOfOptionalHeader', 'FileHeader.TimeDateStamp', 'GROUP.DLL', 'HYPERTERM.DLL', 'KERNL32.DLL', 'MAPI.DLL', 'MFC30.DLL', 'MSFS32.DLL', 'MSNDUI.DLL', 'MSNET32.DLL', 'MSPST32.DLL', 'MSSHRUI.DLL', 'MSVIEWUT.DLL', 'NAL.DLL']
# peminer_header_150 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers', 'OptionalHeader.LoaderFlags', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.Magic', 'Section.text.PointerToRelocations', 'WINSOCK.DLL', 'Section.text.NumberOfLinenumbers', 'Section.rsrc.PointerToRelocations', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'Section.rsrc.PointerToLinenumbers', 'Section.text.NumberOfRelocations', 'Section.rsrc.NumberOfRelocations', 'Section.data.PointerToLinenumbers', 'MAPI32.DLL', 'Section.data.PointerToRelocations', 'Section.data.NumberOfLinenumbers', 'DCIMAN32.DLL', 'ADVAP132.DLL', 'AWFAXP32.DLL', 'AWFXAB32.DLL', 'AWPWD32.DLL', 'AWRESX32.DLL', 'AWUTIL32.DLL', 'BHNETB.DLL', 'BHSUPP.DLL', 'CCAPI.DLL', 'CCEI.DLL', 'CCPSH.DLL', 'CCTN20.DLL', 'CMC.DLL', 'DCIMAN.DLL', 'DSKMAINT.DLL', 'FileHeader.Machine', 'FileHeader.NumberOfSections', 'FileHeader.NumberOfSymbols', 'FileHeader.PointerToSymbolTable', 'FileHeader.SizeOfOptionalHeader', 'FileHeader.TimeDateStamp', 'GROUP.DLL', 'HYPERTERM.DLL', 'KERNL32.DLL', 'MAPI.DLL', 'MFC30.DLL', 'MSFS32.DLL', 'MSNDUI.DLL', 'MSNET32.DLL', 'MSPST32.DLL', 'MSSHRUI.DLL', 'MSVIEWUT.DLL', 'NAL.DLL', 'NDIS30.DLL', 'NETAPI.DLL', 'NETBIOS.DLL', 'NETDI.DLL', 'NETSETUP.DLL', 'NWAB32.DLL', 'NWNET32.DLL', 'NWNP32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.VirtualAddress', 'POWERCFG.DLL']
# peminer_header_160 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers', 'OptionalHeader.LoaderFlags', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.Magic', 'Section.text.PointerToRelocations', 'WINSOCK.DLL', 'Section.text.NumberOfLinenumbers', 'Section.rsrc.PointerToRelocations', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'Section.rsrc.PointerToLinenumbers', 'Section.text.NumberOfRelocations', 'Section.rsrc.NumberOfRelocations', 'Section.data.PointerToLinenumbers', 'MAPI32.DLL', 'Section.data.PointerToRelocations', 'Section.data.NumberOfLinenumbers', 'DCIMAN32.DLL', 'ADVAP132.DLL', 'AWFAXP32.DLL', 'AWFXAB32.DLL', 'AWPWD32.DLL', 'AWRESX32.DLL', 'AWUTIL32.DLL', 'BHNETB.DLL', 'BHSUPP.DLL', 'CCAPI.DLL', 'CCEI.DLL', 'CCPSH.DLL', 'CCTN20.DLL', 'CMC.DLL', 'DCIMAN.DLL', 'DSKMAINT.DLL', 'FileHeader.Machine', 'FileHeader.NumberOfSections', 'FileHeader.NumberOfSymbols', 'FileHeader.PointerToSymbolTable', 'FileHeader.SizeOfOptionalHeader', 'FileHeader.TimeDateStamp', 'GROUP.DLL', 'HYPERTERM.DLL', 'KERNL32.DLL', 'MAPI.DLL', 'MFC30.DLL', 'MSFS32.DLL', 'MSNDUI.DLL', 'MSNET32.DLL', 'MSPST32.DLL', 'MSSHRUI.DLL', 'MSVIEWUT.DLL', 'NAL.DLL', 'NDIS30.DLL', 'NETAPI.DLL', 'NETBIOS.DLL', 'NETDI.DLL', 'NETSETUP.DLL', 'NWAB32.DLL', 'NWNET32.DLL', 'NWNP32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.VirtualAddress', 'POWERCFG.DLL', 'RASAPI16.DLL', 'RASPI.DLL', 'RPCLTC1.DLL', 'RPCNS4.DLL', 'RPCTLC3.DLL', 'RPCTLC5.DLL', 'RPCTLC6.DLL', 'RPCTLS3.DLL', 'RPCTLS5.DLL', 'RPCTLS6.DLL']
# peminer_header_170 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers', 'OptionalHeader.LoaderFlags', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.Magic', 'Section.text.PointerToRelocations', 'WINSOCK.DLL', 'Section.text.NumberOfLinenumbers', 'Section.rsrc.PointerToRelocations', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'Section.rsrc.PointerToLinenumbers', 'Section.text.NumberOfRelocations', 'Section.rsrc.NumberOfRelocations', 'Section.data.PointerToLinenumbers', 'MAPI32.DLL', 'Section.data.PointerToRelocations', 'Section.data.NumberOfLinenumbers', 'DCIMAN32.DLL', 'ADVAP132.DLL', 'AWFAXP32.DLL', 'AWFXAB32.DLL', 'AWPWD32.DLL', 'AWRESX32.DLL', 'AWUTIL32.DLL', 'BHNETB.DLL', 'BHSUPP.DLL', 'CCAPI.DLL', 'CCEI.DLL', 'CCPSH.DLL', 'CCTN20.DLL', 'CMC.DLL', 'DCIMAN.DLL', 'DSKMAINT.DLL', 'FileHeader.Machine', 'FileHeader.NumberOfSections', 'FileHeader.NumberOfSymbols', 'FileHeader.PointerToSymbolTable', 'FileHeader.SizeOfOptionalHeader', 'FileHeader.TimeDateStamp', 'GROUP.DLL', 'HYPERTERM.DLL', 'KERNL32.DLL', 'MAPI.DLL', 'MFC30.DLL', 'MSFS32.DLL', 'MSNDUI.DLL', 'MSNET32.DLL', 'MSPST32.DLL', 'MSSHRUI.DLL', 'MSVIEWUT.DLL', 'NAL.DLL', 'NDIS30.DLL', 'NETAPI.DLL', 'NETBIOS.DLL', 'NETDI.DLL', 'NETSETUP.DLL', 'NWAB32.DLL', 'NWNET32.DLL', 'NWNP32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.VirtualAddress', 'POWERCFG.DLL', 'RASAPI16.DLL', 'RASPI.DLL', 'RPCLTC1.DLL', 'RPCNS4.DLL', 'RPCTLC3.DLL', 'RPCTLC5.DLL', 'RPCTLC6.DLL', 'RPCTLS3.DLL', 'RPCTLS5.DLL', 'RPCTLS6.DLL', 'RSRC32.DLL', 'Resource.Characteristics', 'Resource.MajorVersion', 'Resource.MinorVersion', 'Resource.NumberOfIdEntries', 'Resource.NumberOfNamedEntries', 'Resource.RT_ACCELERATOR', 'Resource.RT_BITMAP', 'Resource.RT_CURSOR', 'Resource.RT_DIALOG']
# peminer_header_180 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers', 'OptionalHeader.LoaderFlags', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.Magic', 'Section.text.PointerToRelocations', 'WINSOCK.DLL', 'Section.text.NumberOfLinenumbers', 'Section.rsrc.PointerToRelocations', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'Section.rsrc.PointerToLinenumbers', 'Section.text.NumberOfRelocations', 'Section.rsrc.NumberOfRelocations', 'Section.data.PointerToLinenumbers', 'MAPI32.DLL', 'Section.data.PointerToRelocations', 'Section.data.NumberOfLinenumbers', 'DCIMAN32.DLL', 'ADVAP132.DLL', 'AWFAXP32.DLL', 'AWFXAB32.DLL', 'AWPWD32.DLL', 'AWRESX32.DLL', 'AWUTIL32.DLL', 'BHNETB.DLL', 'BHSUPP.DLL', 'CCAPI.DLL', 'CCEI.DLL', 'CCPSH.DLL', 'CCTN20.DLL', 'CMC.DLL', 'DCIMAN.DLL', 'DSKMAINT.DLL', 'FileHeader.Machine', 'FileHeader.NumberOfSections', 'FileHeader.NumberOfSymbols', 'FileHeader.PointerToSymbolTable', 'FileHeader.SizeOfOptionalHeader', 'FileHeader.TimeDateStamp', 'GROUP.DLL', 'HYPERTERM.DLL', 'KERNL32.DLL', 'MAPI.DLL', 'MFC30.DLL', 'MSFS32.DLL', 'MSNDUI.DLL', 'MSNET32.DLL', 'MSPST32.DLL', 'MSSHRUI.DLL', 'MSVIEWUT.DLL', 'NAL.DLL', 'NDIS30.DLL', 'NETAPI.DLL', 'NETBIOS.DLL', 'NETDI.DLL', 'NETSETUP.DLL', 'NWAB32.DLL', 'NWNET32.DLL', 'NWNP32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.VirtualAddress', 'POWERCFG.DLL', 'RASAPI16.DLL', 'RASPI.DLL', 'RPCLTC1.DLL', 'RPCNS4.DLL', 'RPCTLC3.DLL', 'RPCTLC5.DLL', 'RPCTLC6.DLL', 'RPCTLS3.DLL', 'RPCTLS5.DLL', 'RPCTLS6.DLL', 'RSRC32.DLL', 'Resource.Characteristics', 'Resource.MajorVersion', 'Resource.MinorVersion', 'Resource.NumberOfIdEntries', 'Resource.NumberOfNamedEntries', 'Resource.RT_ACCELERATOR', 'Resource.RT_BITMAP', 'Resource.RT_CURSOR', 'Resource.RT_DIALOG', 'Resource.RT_DLGINCLUDE', 'Resource.RT_FONT', 'Resource.RT_FONTDIR', 'Resource.RT_GROUP_CURSOR', 'Resource.RT_GROUP_ICON', 'Resource.RT_ICON', 'Resource.RT_MENU', 'Resource.RT_MESSAGETABLE', 'Resource.RT_RCDATA', 'Resource.RT_STRING']
## 원래 헤더(188)
# peminer_header = ['ADVAP132.DLL', 'AWFAXP32.DLL', 'AWFXAB32.DLL', 'AWPWD32.DLL', 'AWRESX32.DLL', 'AWUTIL32.DLL', 'BHNETB.DLL', 'BHSUPP.DLL', 'CCAPI.DLL', 'CCEI.DLL', 'CCPSH.DLL', 'CCTN20.DLL', 'CMC.DLL', 'COMCTL32.DLL', 'COMDLG32.DLL', 'CRTDLL.DLL', 'DCIMAN.DLL', 'DCIMAN32.DLL', 'DSKMAINT.DLL', 'FileHeader.Characteristics', 'FileHeader.Machine', 'FileHeader.NumberOfSections', 'FileHeader.NumberOfSymbols', 'FileHeader.PointerToSymbolTable', 'FileHeader.SizeOfOptionalHeader', 'FileHeader.TimeDateStamp', 'GDI32.DLL', 'GROUP.DLL', 'HYPERTERM.DLL', 'KERNL32.DLL', 'LZ32.DLL', 'MAPI.DLL', 'MAPI32.DLL', 'MFC30.DLL', 'MPR.DLL', 'MSFS32.DLL', 'MSNDUI.DLL', 'MSNET32.DLL', 'MSPST32.DLL', 'MSSHRUI.DLL', 'MSVIEWUT.DLL', 'NAL.DLL', 'NDIS30.DLL', 'NETAPI.DLL', 'NETAPI32.DLL', 'NETBIOS.DLL', 'NETDI.DLL', 'NETSETUP.DLL', 'NWAB32.DLL', 'NWNET32.DLL', 'NWNP32.DLL', 'OLEDLG.DLL', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.BaseOfCode', 'OptionalHeader.BaseOfData', 'OptionalHeader.CheckSum', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'OptionalHeader.FileAlignment', 'OptionalHeader.ImageBase', 'OptionalHeader.LoaderFlags', 'OptionalHeader.Magic', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.MajorLinkerVersion', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.MinorLinkerVersion', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'OptionalHeader.SectionAlignment', 'OptionalHeader.SizeOfCode', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.SizeOfHeapReserve', 'OptionalHeader.SizeOfImage', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.SizeOfStackReserve', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.Subsystem', 'POWERCFG.DLL', 'RASAPI16.DLL', 'RASAPI32.DLL', 'RASPI.DLL', 'RPCLTC1.DLL', 'RPCNS4.DLL', 'RPCRT4.DLL', 'RPCTLC3.DLL', 'RPCTLC5.DLL', 'RPCTLC6.DLL', 'RPCTLS3.DLL', 'RPCTLS5.DLL', 'RPCTLS6.DLL', 'RSRC32.DLL', 'Resource.Characteristics', 'Resource.MajorVersion', 'Resource.MinorVersion', 'Resource.NumberOfIdEntries', 'Resource.NumberOfNamedEntries', 'Resource.RT_ACCELERATOR', 'Resource.RT_BITMAP', 'Resource.RT_CURSOR', 'Resource.RT_DIALOG', 'Resource.RT_DLGINCLUDE', 'Resource.RT_FONT', 'Resource.RT_FONTDIR', 'Resource.RT_GROUP_CURSOR', 'Resource.RT_GROUP_ICON', 'Resource.RT_ICON', 'Resource.RT_MENU', 'Resource.RT_MESSAGETABLE', 'Resource.RT_RCDATA', 'Resource.RT_STRING', 'Resource.RT_VERSION', 'Resource.TimeDateStamp', 'SAPNSP.DLL', 'SECUR32.DLL', 'SHELL32.DLL', 'SHLWAPI.DLL', 'SLENH.DLL', 'Section.data.Characteristics', 'Section.data.NumberOfLinenumbers', 'Section.data.NumberOfRelocations', 'Section.data.PointerToLinenumbers', 'Section.data.PointerToRawData', 'Section.data.PointerToRelocations', 'Section.data.SizeOfRawData', 'Section.data.VirtualAddress', 'Section.data.VirtualSize', 'Section.rsrc.Characteristics', 'Section.rsrc.NumberOfLinenumbers', 'Section.rsrc.NumberOfRelocations', 'Section.rsrc.PointerToLinenumbers', 'Section.rsrc.PointerToRawData', 'Section.rsrc.PointerToRelocations', 'Section.rsrc.SizeOfRawData', 'Section.rsrc.VirtualAddress', 'Section.rsrc.VirtualSize', 'Section.text.Characteristics', 'Section.text.NumberOfLinenumbers', 'Section.text.NumberOfRelocations', 'Section.text.PointerToLinenumbers', 'Section.text.PointerToRawData', 'Section.text.PointerToRelocations', 'Section.text.SizeOfRawData', 'Section.text.VirtualAddress', 'Section.text.VirtualSize', 'UMDM32.DLL', 'USER32.DLL', 'VERSION.DLL', 'WININET.DLL', 'WINMM.DLL', 'WINREG.DLL', 'WINSOCK.DLL', 'WS2.DLL', 'WSOCK32.DLL']

class PeminerParser:
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    
    def process_report(self):
        # 원래 코드
        # self.vector = [value for _, value in sorted(self.report.items(), key=lambda x: x[0])]
        # 바뀐 코드
        for _,value in sorted(self.report.items(), key=lambda x: x[0]):
          if _ in peminer_header_40:
            self.vector.append(value)
        return self.vector

class PeminerParser:
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    
    def process_report(self):
        # 원래 코드
        # self.vector = [value for _, value in sorted(self.report.items(), key=lambda x: x[0])]
        # 바뀐 코드
        for _,value in sorted(self.report.items(), key=lambda x: x[0]):
          if _ in peminer_header_40:
            self.vector.append(value)
        return self.vector



> #### EMBER



In [ ]:
class EmberParser:
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    
    def get_histogram_info(self):
        histogram = np.array(self.report["histogram"])
        total = histogram.sum()
        vector = histogram / total
        return vector.tolist()
    
    def get_byteentropy_info(self):
        byteentropy = np.array(self.report["byteentropy"])
        total = byteentropy.sum()
        vector = byteentropy / total
        return vector.tolist()

    def get_string_info(self):
        strings = self.report["strings"]

        hist_divisor = float(strings['printables']) if strings['printables'] > 0 else 1.0
        vector = [
            strings['numstrings'], 
            strings['avlength'], 
            strings['printables'],
            strings['entropy'], 
            strings['paths'], 
            strings['urls'],
            strings['registry'], 
            strings['MZ']
        ]
        vector += (np.asarray(strings['printabledist']) / hist_divisor).tolist()
        return vector
    
    def get_general_file_info(self):
        general = self.report["general"]
        vector = [
             general['exports'], general['has_debug'], general['has_relocations'], general['has_resources'],
            general['has_signature'], general['has_tls'], general['size']
        ]
        return vector

    def get_imports_info(self):
        imports = self.report["imports"]
        libraries = list(set([l.lower() for l in imports.keys()]))
        libraries_hashed = FeatureHasher(256, input_type="string").transform([libraries]).toarray()[0]
        import_ = [lib.lower() + ':' + e for lib, elist in imports.items() for e in elist]
        imports_hashed = FeatureHasher(1024, input_type="string").transform([import_]).toarray()[0]
        vector = np.hstack([libraries_hashed, imports_hashed])
        return vector.tolist()

    def get_datadirectories_info(self):
        # 특정 데이터만 가져오기 위해 중요도가 높은 데이터의 index만 Data_Directory 리스트에 저장
        Data_Directory = [0, 1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 13, 19, 20, 21, 24, 25, 27, 29]
        datadirectories = self.report["datadirectories"]
        name_order = ["EXPORT_TABLE", "IMPORT_TABLE", "RESOURCE_TABLE", "EXCEPTION_TABLE", "CERTIFICATE_TABLE",
                      "BASE_RELOCATION_TABLE", "DEBUG", "ARCHITECTURE", "GLOBAL_PTR", "TLS_TABLE", "LOAD_CONFIG_TABLE",
                      "BOUND_IMPORT", "IAT", "DELAY_IMPORT_DESCRIPTOR", "CLR_RUNTIME_HEADER"]
        vector = np.zeros(19, dtype=np.float32)
        cnt = 0
        for i in Data_Directory:
          if i < len(datadirectories):
              if i%2 == 0:
                vector[cnt] = datadirectories[i//2]["size"]
              if i%2 == 1:
                vector[cnt] = datadirectories[i//2]["virtual_address"]
          cnt += 1
        return vector.tolist()

    def get_section_info(self):
        # 특정 데이터만 가져오기 위해 중요도가 높은 데이터의 index만 아래의 리스트에 저장      
        Section_General = [1, 3, 4]
        Section_Size = [1, 4, 5, 14, 22, 28, 32, 35, 41, 42, 43]
        Section_Entropy = [5, 14, 20, 28, 32, 35, 40, 41, 42, 43]
        Section_Virtual_Size = [0, 4, 5, 10, 14, 16, 27, 32, 34, 35, 38, 41, 42, 43]
        Section_Entry = [9, 17, 20, 39]
        Section_Characteristic = [13, 23, 37, 47]

        sections = self.report["section"]
        vector = [
            #0. len(sections),  # total number of sections
            sum(1 for s in sections['sections'] if s['size'] == 0), # number of sections with nonzero size
            #2. sum(1 for s in sections['sections'] if s['name'] == ""),  # number of sections with an empty name
            sum(1 for s in sections['sections'] if 'MEM_READ' in s['props'] and 'MEM_EXECUTE' in s['props']), # number of RX
            sum(1 for s in sections['sections'] if 'MEM_WRITE' in s['props']) # number of W
        ]
        # gross characteristics of each section
        section_sizes = [(s['name'], s['size']) for s in sections['sections']]
        section_sizes_hashed = FeatureHasher(50, input_type="pair").transform([section_sizes]).toarray()[0] # 50 
        section_sizes_hashed_new = []
        for value in Section_Size:
          section_sizes_hashed_new.append(section_sizes_hashed[value])
        
        section_entropy = [(s['name'], s['entropy']) for s in sections['sections']]
        section_entropy_hashed = FeatureHasher(50, input_type="pair").transform([section_entropy]).toarray()[0] # 50
        section_entropy_hashed_new = []
        for value in Section_Entropy:
          section_entropy_hashed_new.append(section_entropy_hashed[value])

        section_vsize = [(s['name'], s['vsize']) for s in sections['sections']]
        section_vsize_hashed = FeatureHasher(50, input_type="pair").transform([section_vsize]).toarray()[0]
        section_vsize_hashed_new = []
        for value in Section_Virtual_Size:
          section_vsize_hashed_new.append(section_vsize_hashed[value])
        
        entry_name_hashed = FeatureHasher(50, input_type="string").transform([sections['entry']]).toarray()[0]
        entry_name_hashed_new = []
        for value in Section_Entry:
          entry_name_hashed_new.append(entry_name_hashed[value])

        characteristics = [p for s in sections['sections'] for p in s['props'] if s['name'] == sections['entry']]
        characteristics_hashed = FeatureHasher(50, input_type="string").transform([characteristics]).toarray()[0]
        characteristics_hashed_new = []
        for value in Section_Characteristic:
          characteristics_hashed_new.append(characteristics_hashed[value])
        
        hstack = np.hstack([
            vector, section_sizes_hashed_new, section_entropy_hashed_new, section_vsize_hashed_new, entry_name_hashed_new,
            characteristics_hashed_new
        ])
        return hstack.tolist()

    def get_exports_info(self):
        exports_hashed = FeatureHasher(128, input_type="string").transform([self.report]).toarray()[0]
        return exports_hashed.tolist()

    def get_header_file_info(self):
        # 특정 데이터만 가져오기 위해 중요도가 높은 데이터의 index만 Header_Info 리스트에 저장      
        Header_Info = [0, 11, 14, 15, 28, 29, 34, 51, 52, 53, 54, 55, 57, 59, 60]
        fh1 = FeatureHasher(10, input_type="string").transform([self.report['header']['coff']['characteristics']]).toarray()[0]
        fh1_new = []
        for i in [0,3,4]:
          fh1_new.append(fh1[i])
        fh2 = FeatureHasher(10, input_type="string").transform([[self.report['header']['optional']['subsystem']]]).toarray()[0]  # 21-30 -> 28, 29
        fh2_new = []
        for i in [7,8]:
          fh2_new.append(fh2[i])
        fh3 = FeatureHasher(10, input_type="string").transform([self.report['header']['optional']['dll_characteristics']]).toarray()[0]  # 31-40 ->34
        fh3_new = []
        fh3_new.append(fh3[3])

        # 리스트에 해당되지 않은 데이터들은 가져올 필요가 없으므로 주석 처리
        hstack = np.hstack([
            self.report['header']['coff']['timestamp'], # 0 
            #FeatureHasher(10, input_type="string").transform([[self.report['header']['coff']['machine']]]).toarray()[0],  # 1~10 
            fh1_new, # 11-20 -> 11,14,15
            fh2_new,  # 21-30 -> 28, 29
            fh3_new,  # 31-40 ->34
            #FeatureHasher(10, input_type="string").transform([[self.report['header']['optional']['magic']]]).toarray()[0],  # 41-50
            self.report['header']['optional']['major_image_version'], # 51
            self.report['header']['optional']['minor_image_version'], # 52
            self.report['header']['optional']['major_linker_version'],# 53
            self.report['header']['optional']['minor_linker_version'],# 54
            self.report['header']['optional']['major_operating_system_version'],# 55
            #self.report['header']['optional']['minor_operating_system_version'],# 56
            self.report['header']['optional']['major_subsystem_version'],# 57
            #self.report['header']['optional']['minor_subsystem_version'],# 58
            self.report['header']['optional']['sizeof_code'],# 59
            self.report['header']['optional']['sizeof_headers'],# 60
            #self.report['header']['optional']['sizeof_heap_commit'],# 61
        ])
        return hstack.tolist()


    def process_report(self):
        vector = []
        vector += self.get_general_file_info()
        #vector += self.get_histogram_info()
        #vector += self.get_byteentropy_info()
        #vector += self.get_string_info()
        vector += self.get_imports_info()
        vector += self.get_datadirectories_info()
        vector += self.get_section_info()
        vector += self.get_exports_info()
        vector += self.get_header_file_info()

        return vector


> #### PESTUDIO



In [ ]:
class PestudioParser:
    def __init__(self, path, flag = 0):
        if flag == -1:
          return
        self.report = read_json(path)
        self.vector = []

    def get_max_entropy_info(self, flag = 0):
        entropy = []
        try:
            if flag == -1:
              raise Exception
            if "entropy" in self.report["image"]["dos-header"]:
              entropy.append(float(self.report["image"]["dos-header"]["entropy"]))
            if "entropy" in self.report["image"]["dos-stub"]:
              entropy.append(float(self.report["image"]["dos-stub"]["entropy"]))
            if "entropy" in self.report["image"]["overlay"]:
              entropy.append(float(self.report["image"]["overlay"]["entropy"]))
            if "entropy" in self.report["image"]["overview"]:
              entropy.append(float(self.report["image"]["overview"]["entropy"]))
            
            Max = max(entropy)

            if Max >= 6.0:
                result = 1
            else:
                result = 0
        except Exception:
            result = -1
        return [result]
       
    def get_certificate_info(self, flag = 0):
        try: 
            if flag == -1:
              raise Exception
            certificate = self.report["image"]["certificate"]

            if certificate == "n/a":
                result = 1
            else:
                result = 0
        except:
            result = -1
        return [result]   

    def get_debug_info(self, flag = 0):
        try: 
            if flag == -1:
              raise Exception
            debug = self.report["image"]["debug"]
            if debug == "n/a":
                result = 1
            else:
                result = 0
        except:
            result = -1
        return [result]        

    def get_network_run_from_swap_info(self, flag = 0):
        try:
            if flag == -1:
              raise Exception
            network_run_from_swap = self.report["image"]["file-header"]["network-run-from-swap"]
            if network_run_from_swap == "true":
                result = 1
            else:
                result = 0
        except: 
            result = -1
        return [result]

    def get_control_flow_guard_info(self, flag = 0):
        try:
            if flag == -1:
              raise Exception
            control_flow_guard = self.report["image"]["optional-header"]["control-flow-guard"]
            if control_flow_guard == "n/a":
                result = 1
            else:
                result = 0
        except:
            result = -1
        return [result]

    def get_exports_info(self, flag = 0):
        try: 
            if flag == -1:
              raise Exception
            exports = self.report["image"]["exports"]
            if exports == "n/a":
                result = 1
            else:
                result = 0
        except:
            result = -1
        return [result]

    def get_tls_callbacks_info(self, flag = 0):
        try: 
            if flag == -1:
              raise Exception
            tls_callbacks = self.report["image"]["tls-callbacks"]
            if tls_callbacks == "n/a":
                result = 1
            else:
                result = 0
        except:
            result = -1
        return [result]

    def get_section_info(self, flag = 0):
        try:
            if flag == -1:
              raise Exception
            sections = self.report["image"]["sections"]["section"]
            num_of_section = len(section)

            max_entropy = 0.0
            min_entropy = 50.0
            excutable = 0
            for section in sections:
                max_entropy = max(max_entropy, float(section["@entropy"]))
                min_entropy = min(min_entropy, float(section["@entropy"]))        

                if section["@excutable"] == "-":
                    excutable += 1
                else:
                    excutable += 0
        except:
            num_of_section = 0
            max_entropy = -1.0
            min_entropy = -1.0
            excutable = -1
        return [num_of_section, max_entropy, min_entropy, excutable]

    def get_import_info(self, flag = 0):
        dic_o = {'VerQueryValueA': 3826, 'GetFileVersionInfoA': 3381, 'GetFileVersionInfoSizeA': 3376, 'lstrcmpiA': 3359, 'HeapReAlloc': 3285, 'FindNextFileA': 3272, 'SetErrorMode': 3241, 'DispatchMessageA': 3241, 'IsWindowEnabled': 3200}
        vector = []
        try:
            if flag == -1:
              raise Exception
            imports = self.report["image"]["imports"]["import"]
            for key,value in dic_o.items():
              # print(key)
              if key in imports["@name"]:
                vector.append(1)
              else:
                vector.append(0)
        except:
            vector = [-1 for i in range(len(dic_o))]
        return vector

    def get_import_info2(self, flag = 0):
        dic_x = {'': 159559, 'RegQueryValueExA': 5448, 'RegOpenKeyExA': 5358, 'SelectObject': 5280, 'GetDeviceCaps': 5019, 'SetTextColor': 4967, 'CharNextA': 4890, 'SetBkColor': 4879, 'FindFirstFileA': 4766, 'SetBkMode': 4735, 'DestroyWindow': 4552, 'GetStartupInfoA': 4496, 'GetFileType': 4437, 'lstrcpynA': 4427, 'ImageList_Create': 4411}
        vector = []
        try:
            if flag == -1:
              raise Exception
            imports = self.report["image"]["imports"]["import"]
            for key,value in dic_x.items():
              # print(key)
              if key in imports["@name"]:
                vector.append(1)
              else:
                vector.append(0)
        except:
            vector = [-1 for i in range(len(dic_x))]
        return vector      
            
    def get_virustotal_info(self, flag = 0):  # 정상 악성 비교해봤을 때, 악성은 offline이 아님
        try:
            if flag == -1:
              raise Exception
            if self.report["image"]["virustotal"] != "offline":
                result = 1
            else:
                result = 0
        except:
            result = -1

        return [result]
        
    def process_report(self, flag = 0):
        vector = []
        vector += self.get_max_entropy_info(flag)
        vector += self.get_certificate_info(flag)
        vector += self.get_debug_info(flag)
        vector += self.get_network_run_from_swap_info(flag)
        vector += self.get_control_flow_guard_info(flag)
        vector += self.get_exports_info(flag)
        vector += self.get_tls_callbacks_info(flag)
        vector += self.get_section_info(flag)
        vector += self.get_import_info(flag)
        vector += self.get_import_info2(flag)
        vector += self.get_virustotal_info(flag)
        return vector

### 학습데이터 구성

> #### PEMINER, EMBER

In [ ]:
import os
import queue

learn_data = []

def get_subdir(path):
    global learn_data
    try:  # 검색이 허가되지 않은 디렉토리 접근에 관한 예외처리
        dirfiles = os.listdir(path)  # path에 해당하는 디렉토리 dirfiles에 추가
    except PermissionError:
        return []

    subdir_list = []
    for each in dirfiles:
        if each.endswith(".json"):  # .txt로 끝나는 파일은 따로 all_txt리스트에 저장
            learn_data.append(path + each)
        full_name = path + each  # path와 each(디렉토리)
        if os.path.isdir(full_name):
            subdir_list.append(full_name + "/")
    return subdir_list


dir_queue = queue.Queue()
# /Library/ 아래의 모든 하위 디렉토리 찾기
dir_queue.put("PEMINER/학습데이터/")  # 검색하고자 하는 디렉토리를 줄 앞에 세움

while not dir_queue.empty():  # 큐가 비어있는 상태인지 확인
    dir_name = dir_queue.get()  # 큐에서 빼낸 dir_name
    subdir_names = get_subdir(dir_name)  # dir_name의 하위 디렉토리를 subdir_names에 저장
    for each in subdir_names:  # subdir_names리스트에 있는 디렉토리들을 다시 큐에 넣음
        dir_queue.put(each)

#for dir in learn_data:
#    print(dir)

> #### PESTUDIO



In [ ]:
import os
import queue

pestudio_data = []

def get_subdir(path):
    global pestudio_data
    try:  # 검색이 허가되지 않은 디렉토리 접근에 관한 예외처리
        dirfiles = os.listdir(path)  # path에 해당하는 디렉토리 dirfiles에 추가
    except PermissionError:
        return []

    subdir_list = []
    for each in dirfiles:
        if each.endswith(".json"):  # .txt로 끝나는 파일은 따로 all_txt리스트에 저장
            pestudio_data.append(path + each)
        full_name = path + each  # path와 each(디렉토리)
        if os.path.isdir(full_name):
            subdir_list.append(full_name + "/")
    return subdir_list


dir_queue = queue.Queue()
# /Library/ 아래의 모든 하위 디렉토리 찾기
dir_queue.put("PESTUDIO/학습데이터/")  # 검색하고자 하는 디렉토리를 줄 앞에 세움

while not dir_queue.empty():  # 큐가 비어있는 상태인지 확인
    dir_name = dir_queue.get()  # 큐에서 빼낸 dir_name
    subdir_names = get_subdir(dir_name)  # dir_name의 하위 디렉토리를 subdir_names에 저장
    for each in subdir_names:  # subdir_names리스트에 있는 디렉토리들을 다시 큐에 넣음
        dir_queue.put(each)

#for dir in pestudio_data:
#    print(dir)

> #### PEMINER + EMBER + PESTUDIO

In [ ]:
# 데이터의 특징 벡터 모음(2차원 리스트) : X
# 데이터의 레이블 모음(1차원 리스트) : y
X, y = [], []

for path in learn_data:
  feature_vector = []
  temp, fdata, fname = path.strip().split("/")
  fname = fname.replace(".json","")
  label = learn_label_table[fname]
  for data in ["PEMINER", "EMBER","PESTUDIO"]:
        path = f"{data}/{fdata}/{fname}.json"
        if data == "PEMINER":
            feature_vector += PeminerParser(path).process_report()
        elif data == "EMBER":
            feature_vector += EmberParser(path).process_report()
        elif data == "PESTUDIO" and path in pestudio_data:
            feature_vector += PestudioParser(path).process_report()
        else:  
            feature_vector += PestudioParser(path, -1).process_report(-1)
  X.append(feature_vector)
  y.append(label)

np.asarray(X).shape, np.asarray(y).shape

((20000, 1571), (20000,))

### 검증데이터 구성

> #### PEMINER, EMBER

In [ ]:
import os
import queue

veri_data = []

def get_subdir(path):
    global veri_data
    try:  # 검색이 허가되지 않은 디렉토리 접근에 관한 예외처리
        dirfiles = os.listdir(path)  # path에 해당하는 디렉토리 dirfiles에 추가
    except PermissionError:
        return []

    subdir_list = []
    for each in dirfiles:
        if each.endswith(".json"):  # .txt로 끝나는 파일은 따로 all_txt리스트에 저장
            veri_data.append(path + each)
        full_name = path + each  # path와 each(디렉토리)
        if os.path.isdir(full_name):
            subdir_list.append(full_name + "/")
    return subdir_list


dir_queue = queue.Queue()
# /Library/ 아래의 모든 하위 디렉토리 찾기
dir_queue.put("PEMINER/검증데이터/")  # 검색하고자 하는 디렉토리를 줄 앞에 세움

while not dir_queue.empty():  # 큐가 비어있는 상태인지 확인
    dir_name = dir_queue.get()  # 큐에서 빼낸 dir_name
    subdir_names = get_subdir(dir_name)  # dir_name의 하위 디렉토리를 subdir_names에 저장
    for each in subdir_names:  # subdir_names리스트에 있는 디렉토리들을 다시 큐에 넣음
        dir_queue.put(each)




> #### PESTUDIO



In [ ]:
import os
import queue

pestudio_veri_data = []

def get_subdir(path):
    global pestudio_veri_data
    try:  # 검색이 허가되지 않은 디렉토리 접근에 관한 예외처리
        dirfiles = os.listdir(path)  # path에 해당하는 디렉토리 dirfiles에 추가
    except PermissionError:
        return []

    subdir_list = []
    for each in dirfiles:
        if each.endswith(".json"):  # .txt로 끝나는 파일은 따로 all_txt리스트에 저장
            pestudio_veri_data.append(path + each)
        full_name = path + each  # path와 each(디렉토리)
        if os.path.isdir(full_name):
            subdir_list.append(full_name + "/")
    return subdir_list

dir_queue = queue.Queue()
# /Library/ 아래의 모든 하위 디렉토리 찾기
dir_queue.put("PESTUDIO/검증데이터/")  # 검색하고자 하는 디렉토리를 줄 앞에 세움

while not dir_queue.empty():  # 큐가 비어있는 상태인지 확인
    dir_name = dir_queue.get()  # 큐에서 빼낸 dir_name
    subdir_names = get_subdir(dir_name)  # dir_name의 하위 디렉토리를 subdir_names에 저장
    for each in subdir_names:  # subdir_names리스트에 있는 디렉토리들을 다시 큐에 넣음
        dir_queue.put(each)

> #### PEMINER + EMBER + PESTUDIO

In [ ]:
# 데이터의 특징 벡터 모음(2차원 리스트) : X
# 데이터의 레이블 모음(1차원 리스트) : y
A, b = [], []

for path in veri_data:
  feature_vector = []
  temp, fdata, fname = path.strip().split("/")
  fname = fname.replace(".json","")
  label = verify_label_table[fname]
  for data in ["PEMINER", "EMBER","PESTUDIO"]:
        path = f"{data}/{fdata}/{fname}.json"
        if data == "PEMINER":
            feature_vector += PeminerParser(path).process_report()
        elif data == "EMBER":
            feature_vector += EmberParser(path).process_report()
        elif data == "PESTUDIO" and path in pestudio_veri_data:
            feature_vector += PestudioParser(path).process_report()
        else:  
            feature_vector += PestudioParser(path, -1).process_report(-1)
  A.append(feature_vector)
  b.append(label)

np.asarray(A).shape, np.asarray(b).shape

((10000, 1571), (10000,))

### 학습 및 검증

In [ ]:
SEED = 41

def load_model(**kwargs):
    if kwargs["model"] == "rf":
        return RandomForestClassifier(random_state=kwargs["random_state"], n_jobs=4)
    elif kwargs["model"] == "dt": # decision tree
        return DecisionTreeClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "lgb":
        return LGBMClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "svm":
        return SVC(random_state=kwargs["random_state"])
    elif kwargs["model"] == "lr":
        return LogisticRegression(random_state=kwargs["random_state"], n_jobs=-1)
    elif kwargs["model"] == "knn":
        return KNeighborsClassifier(n_jobs=-1)
    elif kwargs["model"] == "adaboost":
        return AdaBoostClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "mlp":
        return MLPClassifier(random_state=kwargs["random_state"])
    else:
        print("Unsupported Algorithm")
        return None

def train(X_train, y_train, model):
    '''
        머신러닝 모델을 선택하여 학습을 진행하는 함수
	
        :param X_train: 학습할 2차원 리스트 특징벡터
        :param y_train: 학습할 1차원 리스트 레이블 벡터
        :param model: 문자열, 선택할 머신러닝 알고리즘
        :return: 학습된 머신러닝 모델 객체
    '''
    clf = load_model(model=model, random_state=SEED)
    clf.fit(X_train, y_train)
    return clf

def evaluate(X_test, y_test, model):
    '''
        학습된 머신러닝 모델로 검증 데이터를 검증하는 함수
	
        :param X_test: 검증할 2차원 리스트 특징 벡터
        :param y_test: 검증할 1차원 리스트 레이블 벡터
        :param model: 학습된 머신러닝 모델 객체
    '''
    predict = model.predict(X_test)
    print("정확도", model.score(X_test, y_test))


In [ ]:
# 학습
models = []
for model in ["rf", "lgb"]:
    clf = train(X, y, model)
    models.append(clf)

# 검증
# 실제 검증 시에는 제공한 검증데이터를 검증에 사용해야 함
for model in models:
    evaluate(A, b, model)

정확도 0.9547
정확도 0.9563


### 앙상블

In [ ]:
def ensemble_result(X, y, models):
    '''
        학습된 모델들의 결과를 앙상블하는 함수
   
        :param X: 검증할 2차원 리스트 특징 벡터
        :param y: 검증할 1차원 리스트 레이블 벡터
        :param models: 1개 이상의 학습된 머신러닝 모델 객체를 가지는 1차원 리스트
    '''
    
    # Soft Voting
    # https://devkor.tistory.com/entry/Soft-Voting-%EA%B3%BC-Hard-Voting
    predicts = []
    for model in models:
        prob = [result for _, result in model.predict_proba(X)]
        predicts.append(prob)
    
    predict = np.mean(predicts, axis=0)
    predict = [1 if x >= 0.5 else 0 for x in predict]  
    print("정확도", accuracy_score(y, predict))

In [ ]:
ensemble_result(A, b, models)

정확도 0.9595


### 테스트데이터 구성

> #### PEMINER, EMBER - Test Data

In [ ]:
import os
import queue

test_data = []

def get_subdir(path):
    global test_data
    try:  # 검색이 허가되지 않은 디렉토리 접근에 관한 예외처리
        dirfiles = os.listdir(path)  # path에 해당하는 디렉토리 dirfiles에 추가
    except PermissionError:
        return []

    subdir_list = []
    for each in dirfiles:
        if each.endswith(".json"):  # .txt로 끝나는 파일은 따로 all_txt리스트에 저장
            test_data.append(path + each)
        full_name = path + each  # path와 each(디렉토리)
        if os.path.isdir(full_name):
            subdir_list.append(full_name + "/")
    return subdir_list


dir_queue = queue.Queue()
# /Library/ 아래의 모든 하위 디렉토리 찾기
dir_queue.put("PEMINER/테스트데이터/")  # 검색하고자 하는 디렉토리를 줄 앞에 세움

while not dir_queue.empty():  # 큐가 비어있는 상태인지 확인
    dir_name = dir_queue.get()  # 큐에서 빼낸 dir_name
    subdir_names = get_subdir(dir_name)  # dir_name의 하위 디렉토리를 subdir_names에 저장
    for each in subdir_names:  # subdir_names리스트에 있는 디렉토리들을 다시 큐에 넣음
        dir_queue.put(each)

> #### PESTUDIO - Test Data

In [ ]:
import os
import queue

pestudio_test_data = []

def get_subdir(path):
    global pestudio_test_data
    try:  # 검색이 허가되지 않은 디렉토리 접근에 관한 예외처리
        dirfiles = os.listdir(path)  # path에 해당하는 디렉토리 dirfiles에 추가
    except PermissionError:
        return []

    subdir_list = []
    for each in dirfiles:
        if each.endswith(".json"):  # .txt로 끝나는 파일은 따로 all_txt리스트에 저장
            pestudio_test_data.append(path + each)
        full_name = path + each  # path와 each(디렉토리)
        if os.path.isdir(full_name):
            subdir_list.append(full_name + "/")
    return subdir_list


dir_queue = queue.Queue()
# /Library/ 아래의 모든 하위 디렉토리 찾기
dir_queue.put("PESTUDIO/테스트데이터/")  # 검색하고자 하는 디렉토리를 줄 앞에 세움

while not dir_queue.empty():  # 큐가 비어있는 상태인지 확인
    dir_name = dir_queue.get()  # 큐에서 빼낸 dir_name
    subdir_names = get_subdir(dir_name)  # dir_name의 하위 디렉토리를 subdir_names에 저장
    for each in subdir_names:  # subdir_names리스트에 있는 디렉토리들을 다시 큐에 넣음
        dir_queue.put(each)

> #### PEMINER + EMBER + PESTUDIO - Test Data

In [ ]:
# 데이터의 특징 벡터 모음(2차원 리스트) : T
T = []
file_name = []
for path in test_data:
  global file_name
  feature_vector = []
  temp, fdata, fname = path.strip().split("/")
  fname = fname.replace(".json","")
  file_name.append(fname)
  for data in ["PEMINER", "EMBER", "PESTUDIO"]:
        path = f"{data}/{fdata}/{fname}.json"
        if data == "PEMINER":
            feature_vector += PeminerParser(path).process_report()
        elif data == "EMBER":
            feature_vector += EmberParser(path).process_report()
        elif data == "PESTUDIO" and path in pestudio_veri_data:
            feature_vector += PestudioParser(path).process_report()
        else:  
            feature_vector += PestudioParser(path, -1).process_report(-1)
  T.append(feature_vector)

np.asarray(T).shape

(10000, 1571)

### 테스트 실행

In [ ]:
def test(X, models):
    predicts = []
    for model in models:
        prob = [result for _, result in model.predict_proba(X)]
        predicts.append(prob)
    
    predict = np.mean(predicts, axis=0)
    predict = [1 if x >= 0.5 else 0 for x in predict]

    return predict    

test_ret = test(T, models)

import csv

f = open('predict.csv', 'w', newline='')
wr = csv.writer(f)
wr.writerow(['file', 'predict'])

for i in range(len(test_ret)):
  wr.writerow([file_name[i], test_ret[i]])

f.close()

---
### PESTUDIO import 함수에서 사용할 리스트 생성
- 주어진 모든 데이터에 대해 함수이름 저장
1.   정상파일 함수 이름 & 빈도수 : func_name_o
2.   악성파일 함수 이름 & 빈도수 : func_name_x

In [ ]:
func_name = {}
for path in pestudio_data:
  temp, fdata, fname = path.strip().split("/")
  fname = fname.replace(".json","")
  path = f"PESTUDIO/{fdata}/{fname}.json"
  data = read_json(path)
  label = learn_label_table[fname]
  if "image" not in data:
    continue
  if "imports" not in data["image"]:
    continue
  if "import" not in data["image"]["imports"]:
    continue
  for pe_import in data["image"]["imports"]["import"]:
    if "@name" not in pe_import:
      continue
    if pe_import == "@name":
      continue
    name = pe_import["@name"]
    if name not in func_name:
      func_name[name] = 1
    else:
      func_name[name] += 1

In [ ]:
for path in pestudio_veri_data:
  temp, fdata, fname = path.strip().split("/")
  fname = fname.replace(".json","")
  path = f"PESTUDIO/{fdata}/{fname}.json"
  data = read_json(path)
  label = verify_label_table[fname]
  if "image" not in data:
    continue
  if "imports" not in data["image"]:
    continue
  if "import" not in data["image"]["imports"]:
    continue
  for pe_import in data["image"]["imports"]["import"]:
    if "@name" not in pe_import:
      continue
    if pe_import == "@name":
      continue
    name = pe_import["@name"]
    if name not in func_name:
      func_name[name] = 1
    else:
      func_name[name] += 1

In [ ]:
func_name_o = {}
for path in pestudio_data:
  temp, fdata, fname = path.strip().split("/")
  fname = fname.replace(".json","")
  path = f"PESTUDIO/{fdata}/{fname}.json"
  data = read_json(path)
  label = learn_label_table[fname]
  if label == 1: # 악성파일인 경우 pass
    continue
  if "image" not in data:
    continue
  if "imports" not in data["image"]:
    continue
  if "import" not in data["image"]["imports"]:
    continue
  for pe_import in data["image"]["imports"]["import"]:
    if "@name" not in pe_import:
      continue
    if pe_import == "@name":
      continue
    name = pe_import["@name"]
    if name not in func_name_o:
      func_name_o[name] = 1
    else:
      func_name_o[name] += 1

In [ ]:
for path in pestudio_veri_data:
  temp, fdata, fname = path.strip().split("/")
  fname = fname.replace(".json","")
  path = f"PESTUDIO/{fdata}/{fname}.json"
  data = read_json(path)
  label = verify_label_table[fname]
  if label == 1: # 악성파일인 경우 pass
    continue
  if "image" not in data:
    continue
  if "imports" not in data["image"]:
    continue
  if "import" not in data["image"]["imports"]:
    continue
  for pe_import in data["image"]["imports"]["import"]:
    if "@name" not in pe_import:
      continue
    if pe_import == "@name":
      continue
    name = pe_import["@name"]
    if name not in func_name_o:
      func_name_o[name] = 1
    else:
      func_name_o[name] += 1

In [ ]:
func_name_x = {}
for path in pestudio_data:
  temp, fdata, fname = path.strip().split("/")
  fname = fname.replace(".json","")
  path = f"PESTUDIO/{fdata}/{fname}.json"
  data = read_json(path)
  label = learn_label_table[fname]
  if label == 0:
    continue
  if "image" not in data:
    continue
  if "imports" not in data["image"]:
    continue
  if "import" not in data["image"]["imports"]:
    continue
  for pe_import in data["image"]["imports"]["import"]:
    if "@name" not in pe_import:
      continue
    if pe_import == "@name":
      continue
    name = pe_import["@name"]
    if name not in func_name_x:
      func_name_x[name] = 1
    else:
      func_name_x[name] += 1

In [ ]:
for path in pestudio_veri_data:
  temp, fdata, fname = path.strip().split("/")
  fname = fname.replace(".json","")
  path = f"PESTUDIO/{fdata}/{fname}.json"
  data = read_json(path)
  label = verify_label_table[fname]
  if label == 0:
    continue
  if "image" not in data:
    continue
  if "imports" not in data["image"]:
    continue
  if "import" not in data["image"]["imports"]:
    continue
  for pe_import in data["image"]["imports"]["import"]:
    if "@name" not in pe_import:
      continue
    if pe_import == "@name":
      continue
    name = pe_import["@name"]
    if name not in func_name_x:
      func_name_x[name] = 1
    else:
      func_name_x[name] += 1

In [ ]:
'''
print(len(func_name))
print(len(func_name_o))
print(len(func_name_x))
func_name_t = sorted(func_name.items(), key=lambda x: x[1], reverse=True)[:50]
func_name_o_t = sorted(func_name_o.items(), key=lambda x: x[1], reverse=True)[:50]
func_name_x_t = sorted(func_name_x.items(), key=lambda x: x[1], reverse=True)[:50]
print(func_name_t)
print(func_name_o_t)
print(func_name_x_t)
'''
func_name = [('', 161684), ('GetProcAddress', 16161), ('ExitProcess', 15221), ('GetLastError', 13670), ('CloseHandle', 13113), ('WriteFile', 12557), ('GetModuleHandleA', 12253), ('MultiByteToWideChar', 12248), ('Sleep', 11176), ('GetCurrentProcess', 11047), ('GetTickCount', 11032), ('RegCloseKey', 11008), ('FreeLibrary', 10928), ('LoadLibraryA', 10780), ('GetCommandLineA', 10275), ('ReadFile', 9958), ('GetModuleFileNameA', 9895), ('GetCurrentThreadId', 9591), ('SetFilePointer', 9481), ('WideCharToMultiByte', 9323), ('FindClose', 9045), ('UnhandledExceptionFilter', 8818), ('VirtualAlloc', 8684), ('lstrlenA', 8421), ('CreateFileA', 8351), ('GetStdHandle', 8017), ('RtlUnwind', 7998), ('GetFileSize', 7953), ('EnterCriticalSection', 7893), ('LeaveCriticalSection', 7861), ('DeleteObject', 7848), ('WaitForSingleObject', 7726), ('VirtualFree', 7674), ('SelectObject', 7534), ('RegQueryValueExA', 7529), ('DeleteCriticalSection', 7479), ('QueryPerformanceCounter', 7467), ('RegOpenKeyExA', 7423), ('CreateThread', 7278), ('RaiseException', 7171), ('GetDeviceCaps', 7153), ('GetCurrentProcessId', 7010), ('SetTextColor', 6923), ('SetBkColor', 6699), ('TerminateProcess', 6682), ('GetDC', 6578), ('GetFileType', 6569), ('SetBkMode', 6569), ('DestroyWindow', 6553), ('GetSystemTimeAsFileTime', 6452), ('CoTaskMemFree', 6431), ('CharNextA', 6363), ('SetUnhandledExceptionFilter', 6352), ('GetVersion', 6225), ('ShowWindow', 6123), ('FindFirstFileA', 6114), ('HeapAlloc', 6088), ('CoCreateInstance', 6071), ('GetACP', 5897), ('GetStartupInfoA', 5879), ('EnableWindow', 5862), ('lstrcpynA', 5835), ('GetWindowRect', 5830), ('GlobalAlloc', 5823), ('HeapFree', 5802), ('MessageBoxA', 5744), ('ImageList_Create', 5728), ('GetClientRect', 5694), ('GetModuleHandleW', 5689), ('CreateBrushIndirect', 5667), ('ImageList_Destroy', 5657), ('GetSystemMetrics', 5624), ('InitializeCriticalSection', 5527), ('PostQuitMessage', 5471), ('wsprintfA', 5442), ('ShellExecuteA', 5420), ('GetDlgItem', 5413), ('TlsGetValue', 5399), ('SetWindowPos', 5370), ('GlobalFree', 5368), ('IsWindow', 5238), ('InterlockedDecrement', 5226), ('LoadLibraryExA', 5219), ('SetForegroundWindow', 5194), ('LocalFree', 5190), ('CreateFontIndirectA', 5103), ('BeginPaint', 5098), ('SetLastError', 5084), ('EndDialog', 5045), ('SetTimer', 5040), ('InterlockedIncrement', 5038), ('GlobalUnlock', 5018), ('EndPaint', 5009), ('GlobalLock', 5004), ('TlsSetValue', 5001), ('GetModuleFileNameW', 4992), ('InvalidateRect', 4983), ('GetSysColor', 4964), ('IsWindowVisible', 4961), ('HeapReAlloc', 4924), ('GetExitCodeProcess', 4890), ('VerQueryValueA', 4835), ('lstrcmpiA', 4825), ('SetErrorMode', 4820), ('SetCursor', 4722), ('CreateFileW', 4690), ('SendMessageA', 4672), ('FillRect', 4668), ('MulDiv', 4632), ('IsWindowEnabled', 4626), ('ScreenToClient', 4585), ('OleInitialize', 4581), ('IsDebuggerPresent', 4566), ('LocalAlloc', 4493), ('SetEndOfFile', 4475), ('GetCPInfo', 4464), ('SetFileTime', 4455), ('FindNextFileA', 4454), ('DispatchMessageA', 4444), ('GetStringTypeW', 4429), ('DeleteFileA', 4421), ('17', 4421), ('GetOEMCP', 4380), ('OleUninitialize', 4375), ('TlsAlloc', 4367), ('lstrcmpA', 4362), ('GetFileVersionInfoA', 4348), ('GetFileVersionInfoSizeA', 4341), ('GetProcessHeap', 4324), ('GetFileAttributesA', 4282), ('LCMapStringW', 4270), ('CreateProcessA', 4223), ('GetEnvironmentStringsW', 4218), ('FreeEnvironmentStringsW', 4192), ('RegSetValueExA', 4187), ('CreateDirectoryA', 4182), ('VirtualProtect', 4173), ('EnableMenuItem', 4163), ('GetMessagePos', 4154), ('DefWindowProcA', 4148), ('TlsFree', 4146), ('CreatePopupMenu', 4144), ('GetLocaleInfoA', 4132), ('GetCommandLineW', 4105), ('CreateWindowExA', 4063), ('ExitWindowsEx', 4033), ('TrackPopupMenu', 4025), ('HeapSize', 4009), ('OpenClipboard', 4005), ('LoadCursorA', 4004), ('CloseClipboard', 3982), ('SHGetSpecialFolderLocation', 3978), ('VirtualQuery', 3959), ('FlushFileBuffers', 3954), ('EmptyClipboard', 3932), ('GetTempPathA', 3907), ('CreateCompatibleDC', 3902), ('CompareFileTime', 3883), ('SetClipboardData', 3873), ('SetFileAttributesA', 3857), ('CheckDlgButton', 3850), ('GetSystemMenu', 3844), ('lstrcatA', 3826), ('RegCreateKeyExA', 3779), ('GetWindowLongA', 3756), ('GetStockObject', 3735), ('GetSystemDirectoryA', 3731), ('SHGetFileInfoA', 3727), ('DeleteDC', 3698), ('PeekMessageA', 3693), ('ImageList_AddMasked', 3680), ('SHFileOperationA', 3663), ('SetWindowLongA', 3615), ('SetStdHandle', 3580), ('CopyFileA', 3565), ('LoadLibraryExW', 3525), ('RegisterClassA', 3503), ('GetWindowsDirectoryA', 3483), ('IsValidCodePage', 3477), ('InitializeCriticalSectionAndSpinCount', 3474), ('BitBlt', 3442), ('HeapCreate', 3425), ('lstrlenW', 3413), ('RegOpenKeyExW', 3407), ('GetPrivateProfileStringA', 3396), ('RemoveDirectoryA', 3388), ('CreateCompatibleBitmap', 3359), ('LoadLibraryW', 3357), ('SystemParametersInfoA', 3349), ('SetWindowTextA', 3346), ('lstrcpyA', 3343), ('RegQueryValueExW', 3340), ('100', 3338), ('WritePrivateProfileStringA', 3337), ('RegDeleteValueA', 3330), ('__vbaExceptHandler', 3298), ('_initterm', 3294), ('CallWindowProcA', 3284), ('RegDeleteKeyA', 3282), ('MoveFileA', 3265), ('CreateSolidBrush', 3241), ('SetCurrentDirectoryA', 3236), ('DialogBoxParamA', 3211), ('DrawTextA', 3208), ('GetTempFileNameA', 3194), ('GetFullPathNameA', 3174), ('ExpandEnvironmentStringsA', 3164), ('LoadBitmapA', 3164), ('EVENT_SINK_QueryInterface', 3154), ('EVENT_SINK_Release', 3153), ('EVENT_SINK_AddRef', 3152), ('GetConsoleMode', 3146), ('SaveDC', 3121), ('ReleaseDC', 3096), ('GetShortPathNameA', 3082), ('SetDlgItemTextA', 3058), ('WriteConsoleW', 3056), ('AppendMenuA', 3049), ('GetDiskFreeSpaceA', 3041), ('RegEnumKeyA', 3040), ('GetClassInfoA', 3021), ('GetThreadLocale', 3017), ('DllFunctionCall', 3016), ('RegEnumValueA', 3006), ('GetConsoleCP', 3004), ('LoadImageA', 2994), ('InterlockedExchange', 2975), ('CreateDialogParamA', 2962), ('GetStartupInfoW', 2961), ('GetDlgItemTextA', 2939), ('SHGetPathFromIDListA', 2938), ('LoadStringA', 2933), ('IsProcessorFeaturePresent', 2928), ('SearchPathA', 2925), ('CharPrevA', 2917), ('DecodePointer', 2910), ('SysAllocStringLen', 2894), ('FindWindowExA', 2885), ('SysFreeString', 2852), ('SHBrowseForFolderA', 2811), ('CreateBitmap', 2805), ('SetClassLongA', 2799), ('SendMessageTimeoutA', 2796), ('free', 2795), ('SetHandleCount', 2787), ('GetVersionExA', 2785), ('EncodePointer', 2742), ('StretchBlt', 2720), ('HeapDestroy', 2719), ('SetViewportOrgEx', 2676), ('DeleteFileW', 2639), ('RestoreDC', 2619), ('FindFirstFileW', 2613), ('GetObjectA', 2603), ('OpenProcessToken', 2556), ('CreateDIBSection', 2541), ('RectVisible', 2524), ('MessageBoxIndirectA', 2515), ('GetKeyboardType', 2499), ('RegSetValueExW', 2492), ('TranslateMessage', 2485), ('MoveToEx', 2472), ('GetFileAttributesW', 2464), ('GetDIBits', 2445), ('RealizePalette', 2444), ('FindNextFileW', 2440), ('SendMessageW', 2429), ('ImageList_Add', 2423), ('GetPixel', 2413), ('SetEvent', 2402), ('GetClipBox', 2401), ('CoUninitialize', 2397), ('LineTo', 2394), ('SelectPalette', 2381), ('_CIsin', 2369), ('_CIcos', 2364), ('_CIsqrt', 2363), ('malloc', 2358), ('RegCreateKeyExW', 2347), ('_allmul', 2341), ('memset', 2334), ('CreatePalette', 2330), ('_CIlog', 2327), ('IntersectClipRect', 2324), ('LoadResource', 2322), ('CoInitialize', 2321), ('_CIexp', 2320), ('PatBlt', 2318), ('ExcludeClipRect', 2310), ('_CItan', 2308), ('_CIatan', 2302), ('SetWindowOrgEx', 2297), ('SysReAllocStringLen', 2293), ('_adj_fdivr_m32i', 2283), ('_adj_fdiv_m16i', 2282), ('_adj_fprem', 2282), ('_adj_fdiv_m32i', 2282), ('_adj_fdiv_m64', 2281), ('_adj_fprem1', 2281), ('_adj_fdivr_m16i', 2281), ('__vbaChkstk', 2281), ('_adj_fdivr_m64', 2281), ('__vbaFPException', 2281), ('_adj_fdivr_m32', 2281), ('_adj_fdiv_r', 2281), ('_adj_fptan', 2280), ('_adj_fdiv_m32', 2280), ('_adj_fpatan', 2280), ('SetStretchBltMode', 2270), ('CreateDirectoryW', 2249), ('CreateDIBitmap', 2241), ('SetPixel', 2235), ('_onexit', 2233), ('GetTextMetricsA', 2228), ('__dllonexit', 2226), ('GetTempPathW', 2224), ('__vbaHresultCheckObj', 2219), ('GetTextExtentPoint32A', 2212), ('__vbaFreeVar', 2212), ('GetSystemPaletteEntries', 2206), ('SetROP2', 2201), ('memcpy', 2196), ('__vbaFreeStr', 2185), ('GetPaletteEntries', 2174), ('GetBitmapBits', 2167), ('UnrealizeObject', 2164), ('DefWindowProcW', 2160), ('CreateWindowExW', 2152), ('ExitThread', 2145), ('LoadCursorW', 2145), ('LCMapStringA', 2131), ('GetStringTypeA', 2130), ('__vbaStrMove', 2123), ('GetWindowOrgEx', 2115), ('GetLocaleInfoW', 2110), ('__vbaFreeVarList', 2108), ('DispatchMessageW', 2101), ('__vbaNew2', 2093), ('CreateProcessW', 2087), ('CompareStringW', 2087), ('GetCurrentPositionEx', 2074), ('SetEnvironmentVariableA', 2068), ('SetBrushOrgEx', 2066), ('CreatePenIndirect', 2062), ('__vbaFreeObj', 2062), ('ImageList_EndDrag', 2060), ('GetSystemInfo', 2059), ('ImageList_DragMove', 2058), ('ImageList_DragEnter', 2057), ('SetWindowLongW', 2052), ('GetVersionExW', 2050), ('GetWindowLongW', 2031), ('SetDIBColorTable', 2019), ('GetDIBColorTable', 2011), ('ImageList_DragLeave', 2008), ('ImageList_BeginDrag', 2007), ('CreateHalftonePalette', 2003), ('SizeofResource', 2002), ('_amsg_exit', 1998), ('608', 1993), ('ImageList_SetBkColor', 1991), ('__vbaSetSystemError', 1989), ('LockResource', 1981), ('9', 1975), ('CharNextW', 1970), ('ShellExecuteW', 1969), ('ImageList_DragShowNolock', 1953), ('CompareStringA', 1951), ('632', 1933), ('GetEnvironmentStrings', 1930), ('OpenProcess', 1924), ('617', 1924), ('ImageList_GetIconSize', 1923), ('ImageList_Remove', 1922), ('ImageList_GetImageCount', 1920), ('GetBrushOrgEx', 1916), ('SetWindowTextW', 1912), ('ImageList_Draw', 1911), ('MaskBlt', 1909), ('ImageList_DrawEx', 1895), ('ImageList_SetIconSize', 1892), ('SetFileAttributesW', 1892), ('6', 1887), ('FreeEnvironmentStringsA', 1885), ('537', 1879), ('GetFullPathNameW', 1878), ('__vbaVarMove', 1877), ('616', 1876), ('GetDCOrgEx', 1875), ('lstrcmpiW', 1874), ('PeekMessageW', 1863), ('ImageList_GetBkColor', 1850), ('RegDeleteValueW', 1849), ('ImageList_Read', 1845), ('GetSystemTime', 1845), ('2', 1844), ('GetLocalTime', 1842), ('598', 1831), ('CreateEventW', 1819), ('_XcptFilter', 1815), ('lstrcpynW', 1814), ('__vbaFreeStrList', 1814), ('ImageList_Write', 1804), ('595', 1793), ('__vbaStrVarMove', 1790), ('wsprintfW', 1774), ('_adjust_fdiv', 1771), ('lstrcmpW', 1768), ('631', 1768), ('GetParent', 1764), ('RegDeleteKeyW', 1760), ('exit', 1759), ('__vbaStrCopy', 1748), ('UpdateWindow', 1743), ('CopyFileW', 1732), ('516', 1730), ('InternetCloseHandle', 1705), ('SetFilePointerEx', 1693), ('__vbaOnError', 1693), ('_except_handler4_common', 1690), ('__vbaStrCat', 1687), ('GetSystemDirectoryW', 1681), ('600', 1681), ('ExpandEnvironmentStringsW', 1680), ('__vbaVarDup', 1680), ('GetCurrentThread', 1678), ('RemoveDirectoryW', 1673), ('GetModuleHandleExW', 1672), ('8', 1649), ('__vbaObjSet', 1648), ('SetFocus', 1647), ('GetUserDefaultLCID', 1632), ('CreateFontIndirectW', 1623), ('MoveFileW', 1618), ('InterlockedCompareExchange', 1608), ('GetTempFileNameW', 1605), ('_lock', 1601), ('_unlock', 1601), ('__vbaI4Var', 1600), ('__vbaStrCmp', 1599), ('lstrcpyW', 1589), ('DialogBoxParamW', 1589), ('UnmapViewOfFile', 1584), ('MessageBoxW', 1580), ('526', 1578), ('645', 1574), ('619', 1564), ('525', 1562), ('607', 1558), ('CallWindowProcW', 1553), ('SetCurrentDirectoryW', 1552), ('FileTimeToSystemTime', 1547), ('529', 1541), ('DrawTextW', 1539), ('SystemParametersInfoW', 1537), ('685', 1532), ('GetPrivateProfileStringW', 1519), ('memmove', 1509), ('717', 1505), ('ResetEvent', 1504), ('GetWindowsDirectoryW', 1501), ('FindResourceW', 1500), ('OutputDebugStringW', 1499), ('LoadIconA', 1493), ('644', 1491), ('SHGetFolderPathA', 1488), ('VerQueryValueW', 1487), ('LoadImageW', 1485), ('MapViewOfFile', 1482), ('_exit', 1480), ('__setusermatherr', 1474), ('520', 1471), ('InternetReadFile', 1463), ('__set_app_type', 1459), ('IsValidLocale', 1457), ('ShellExecuteExA', 1456), ('send', 1453), ('SetDlgItemTextW', 1451), ('RegisterClassW', 1445), ('__vbaLenBstr', 1429), ('__vbaStrToAnsi', 1424), ('__vbaObjSetAddref', 1417), ('RegEnumValueW', 1416), ('GetDesktopWindow', 1413), ('GetMessageA', 1410), ('LoadBitmapW', 1407), ('4', 1405), ('__vbaFreeObjList', 1395), ('InitCommonControls', 1392), ('581', 1386), ('528', 1385), ('GetForegroundWindow', 1362), ('GetFileVersionInfoW', 1356), ('WritePrivateProfileStringW', 1355), ('518', 1354), ('GetFileVersionInfoSizeW', 1350), ('KillTimer', 1347), ('__vbaStrVarVal', 1345), ('FindWindowExW', 1340), ('__vbaAryDestruct', 1337), ('__vbaErrorOverflow', 1335), ('SHGetPathFromIDListW', 1334), ('593', 1334), ('ExtTextOutA', 1333), ('InternetOpenA', 1328), ('GdipFree', 1327), ('DeleteEnhMetaFile', 1325), ('__vbaVarTstEq', 1317), ('__vbaStrToUnicode', 1317), ('ReleaseMutex', 1314), ('GetWindow', 1312), ('__vbaVarCat', 1312), ('PlayEnhMetaFile', 1311), ('LoadStringW', 1310), ('lstrcatW', 1305), ('GetWinMetaFileBits', 1299), ('PostMessageW', 1295), ('716', 1294), ('618', 1282), ('GetEnhMetaFileHeader', 1281), ('__vbaFpI4', 1274), ('CoTaskMemAlloc', 1271), ('SHGetFileInfoW', 1270), ('GetRgnBox', 1269), ('FormatMessageW', 1269), ('SetEnhMetaFileBits', 1265), ('__p__commode', 1264), ('RegEnumKeyW', 1264), ('__vbaEnd', 1263), ('GetEnhMetaFileBits', 1261), ('StringFromCLSID', 1261), ('__vbaVarAdd', 1257), ('SetWinMetaFileBits', 1255), ('GetShortPathNameW', 1255), ('void __cdecl terminate(void)', 1255), ('GetEnhMetaFilePaletteEntries', 1251), ('__p__fmode', 1251), ('GetDlgItemTextW', 1249), ('648', 1246), ('__vbaFileOpen', 1241), ('PostMessageA', 1241), ('594', 1240), ('_initterm_e', 1240), ('570', 1234), ('__vbaFileClose', 1234), ('CopyEnhMetaFileA', 1233), ('712', 1233), ('GetWindowThreadProcessId', 1232), ('FormatMessageA', 1227), ('SHBrowseForFolderW', 1222), ('GetConsoleOutputCP', 1220), ('711', 1218), ('WSAStartup', 1217), ('atoi', 1214), ('WSACleanup', 1210), ('AdjustTokenPrivileges', 1209), ('546', 1204), ('GetTimeZoneInformation', 1196), ('606', 1194), ('htons', 1189), ('AppendMenuW', 1188), ('socket', 1186), ('WSAIoctl', 1185), ('GetClassInfoW', 1180), ('connect', 1178), ('LookupPrivilegeValueA', 1178), ('SHFileOperationW', 1176), ('GetFocus', 1175), ('closesocket', 1173), ('SearchPathW', 1170), ('URLDownloadToFileA', 1166), ('__vbaInStr', 1161), ('_TrackMouseEvent', 1156), ('Rectangle', 1156), ('inet_addr', 1154), ('SendMessageTimeoutW', 1151), ('GetDiskFreeSpaceW', 1147), ('recv', 1146), ('GetCurrentDirectoryW', 1142), ('GetEnvironmentVariableA', 1139), ('CreateDialogParamW', 1137), ('ImageList_ReplaceIcon', 1135), ('_except_handler3', 1134), ('GetCursorPos', 1134), ('WriteConsoleA', 1128), ('gethostbyname', 1126), ('InternetConnectA', 1120), ('SetEnvironmentVariableW', 1118), ('GetTokenInformation', 1110), ('FindResourceA', 1103), ('GdiFlush', 1100), ('GdiplusStartup', 1100), ('Netbios', 1098), ('_crt_debugger_hook', 1095), ('CreateDCA', 1091), ('__vbaRedim', 1091), ('__vbaAryUnlock', 1091), ('InitializeSListHead', 1088), ('12', 1085), ('GdipDisposeImage', 1082), ('DuplicateHandle', 1079), ('__vbaExitProc', 1078), ('__getmainargs', 1077), ('DragQueryFileA', 1076), ('SetClassLongW', 1074), ('__CxxFrameHandler3', 1074), ('waveInOpen', 1072), ('GetObjectW', 1072), ('_controlfp', 1067), ('CharPrevW', 1060), ('ImageList_GetDragImage', 1058), ('_cexit', 1058), ('acmStreamSize', 1056), ('inet_ntoa', 1051), ('535', 1050), ('capGetDriverDescriptionA', 1042), ('SystemTimeToFileTime', 1042), ('FtpPutFileA', 1040), ('InternetOpenUrlA', 1039), ('GdiplusShutdown', 1038), ('MoveWindow', 1037), ('561', 1036), ('CreateEventA', 1035), ('__vbaI2Var', 1030), ('7', 1027), ('ntohs', 1026), ('519', 1019), ('681', 1019), ('23', 1018), ('__vbaVarTstNe', 1009), ('GetCurrentDirectoryA', 1007), ('SHGetMalloc', 1006), ('WSAGetLastError', 999), ('DisableThreadLibraryCalls', 997), ('WaitForMultipleObjects', 997), ('MethCallEngine', 991), ('MapWindowPoints', 990), ('LoadIconW', 990), ('576', 989), ('CopyRect', 985), ('OutputDebugStringA', 984), ('__vbaVarSub', 983), ('FreeSid', 982), ('bind', 978), ('16', 978), ('GdipDeleteGraphics', 976), ('_CxxThrowException', 976), ('573', 974), ('GdipAlloc', 973), ('__vbaGenerateBoundsError', 969), ('VariantClear', 969), ('_purecall', 966), ('ImageList_SetDragCursorImage', 965), ('115', 963), ('GetKeyState', 956), ('578', 954), ('ResumeThread', 952), ('ProcCallEngine', 950), ('MessageBoxIndirectW', 949), ('listen', 948), ('accept', 947), ('10', 945), ('571', 944), ('ClientToScreen', 942), ('CharUpperW', 942), ('GetMessageW', 941), ('__vbaAryConstruct2', 940), ('CreateStreamOnHGlobal', 939), ('580', 936), ('IsDlgButtonChecked', 932), ('CreateMutexA', 932), ('__vbaBoolVarNull', 930), ('AllocateAndInitializeSid', 930), ('strncmp', 928), ('FileTimeToLocalFileTime', 925), ('acmStreamOpen', 916), ('PtInRect', 916), ('IsIconic', 916), ('GetActiveWindow', 913), ('ioctlsocket', 912), ('GetWindowTextA', 910), ('SetThreadPriority', 908), ('gethostname', 906), ('CreateFileMappingW', 906), ('select', 904), ('610', 903), ('__vbaObjVar', 902), ('19', 899), ('3', 898), ('GdipCreateBitmapFromScan0', 895), ('GdipGetImageGraphicsContext', 893), ('321', 888), ('sprintf', 888), ('IsEqualGUID', 883), ('PlaySoundA', 877), ('OffsetRect', 873), ('660', 872), ('CreateMutexW', 871), ('__CppXcptFilter', 869), ('__vbaVarCopy', 867), ('sendto', 864), ('mciSendStringA', 864), ('GdipDrawImageRectI', 864), ('CLSIDFromString', 863), ('__vbaStrVarCopy', 863), ('GdipGetImagePixelFormat', 861), ('VariantChangeTypeEx', 861), ('getservbyname', 859), ('getsockname', 859), ('320', 859), ('MoveFileExW', 857), ('ReleaseCapture', 857), ('SetMapMode', 857), ('shutdown', 853), ('FindResourceExW', 852), ('HeapSetInformation', 851), ('GetDriveTypeA', 851), ('void __cdecl operator delete(void *)', 850), ('GdipSetInterpolationMode', 849), ('612', 849), ('acmStreamConvert', 848), ('DestroyIcon', 848), ('GdipGetImageEncoders', 845), ('GdipGetImageEncodersSize', 845), ('__vbaI2I4', 841), ('SetRect', 840), ('SetCapture', 838), ('613', 837), ('__vbaInStrVar', 836), ('CreateRectRgn', 835), ('IsBadReadPtr', 835), ('ReadConsoleW', 832), ('__vbaUbound', 831), ('601', 829), ('waveInAddBuffer', 825), ('waveInUnprepareHeader', 824), ('GetWindowTextW', 824), ('gethostbyaddr', 823), ('waveInClose', 823), ('RegEnumKeyExW', 823), ('InitCommonControlsEx', 821), ('UnregisterClassA', 821), ('waveInStart', 820), ('waveInPrepareHeader', 820), ('strlen', 820), ('__vbaLateMemCall', 819), ('waveInReset', 818), ('VariantCopyInd', 818), ('SysStringLen', 818), ('GdipSaveImageToStream', 817), ('GetAsyncKeyState', 817), ('599', 817), ('__vbaLsetFixstr', 817), ('__vbaVarTstGt', 816), ('563', 814), ('__clean_type_info_names_internal', 814), ('_malloc_crt', 810), ('666', 809), ('acmStreamClose', 808), ('GetSubMenu', 808), ('acmStreamUnprepareHeader', 804), ('acmStreamPrepareHeader', 804), ('CoInitializeEx', 797), ('SHGetFolderPathW', 797), ('acmStreamReset', 790), ('EVENT_SINK_GetIDsOfNames', 787), ('__vbaRecAnsiToUni', 783), ('__vbaRecUniToAnsi', 783), ('EVENT_SINK_Invoke', 782), ('Zombie_GetTypeInfo', 782), ('Zombie_GetTypeInfoCount', 782), ('319', 782), ('strncpy', 781), ('HttpQueryInfoA', 778), ('CreateToolhelp32Snapshot', 778), ('_acmdln', 777), ('__vbaRedimPreserve', 775), ('579', 770), ('DestroyMenu', 765), ('__vbaVarCmpEq', 763), ('EnumSystemLocalesA', 758), ('void * __cdecl operator new(unsigned int)', 755), ('GdipCreateBitmapFromHBITMAP', 754), ('531', 753), ('CreateFileMappingA', 750), ('GetTextExtentPoint32W', 749), ('CLSIDFromProgID', 749), ('RegisterClassExW', 748), ('ShellExecuteExW', 748), ('NtQuerySystemInformation', 746), ('MsgWaitForMultipleObjects', 746), ('UnhookWindowsHookEx', 745), ('__WSAFDIsSet', 740), ('527', 740), ('LocalFileTimeToFileTime', 737), ('RegOpenKeyA', 737), ('SetActiveWindow', 736), ('__vbaPrintFile', 735), ('__vbaAryLock', 730), ('EnumWindows', 728), ('StringFromGUID2', 722), ('_wcsicmp', 720), ('__vbaLenVar', 719), ('realloc', 719), ('CallNextHookEx', 718), ('RegisterClassExA', 717), ('714', 717), ('LookupPrivilegeValueW', 717), ('__vbaLateMemCallLd', 712), ('__vbaVarAnd', 709), ('__vbaFpI2', 709), ('626', 707), ('__vbaStrI4', 707), ('GetMenu', 705), ('WaitForSingleObjectEx', 705), ('EqualSid', 702), ('GetWindowDC', 698), ('GetSysColorBrush', 698), ('ReleaseSemaphore', 698), ('IsDBCSLeadByte', 697), ('CloseServiceHandle', 697), ('DeviceIoControl', 696), ('__vbaVarLateMemCallLd', 694), ('strchr', 694), ('__vbaVarVargNofree', 693), ('709', 692), ('CreatePen', 689), ('558', 688), ('RegQueryInfoKeyW', 687), ('__vbaStrI2', 684), ('_stricmp', 680), ('SendDlgItemMessageA', 679), ('TerminateThread', 679), ('fclose', 679), ('GetCapture', 677), ('116', 677), ('689', 676), ('GetFileTime', 673), ('public: virtual __thiscall type_info::~type_info(void)', 673), ('592', 672), ('690', 671), ('EnumSystemLocalesW', 671), ('AdjustWindowRectEx', 669), ('TextOutA', 665), ('SelectClipRgn', 663), ('__vbaFixstrConstruct', 663), ('MessageBeep', 662), ('CryptReleaseContext', 662), ('__vbaVarOr', 662), ('RegisterWindowMessageA', 660), ('GetLastActivePopup', 659), ('GetDlgCtrlID', 656), ('__vbaLateIdCallLd', 655), ('GetMenuItemCount', 654), ('IsBadWritePtr', 652), ('710', 650), ('DrawFocusRect', 645), ('GetUserNameA', 644), ('LocalReAlloc', 641), ('572', 641), ('18', 638), ('542', 636), ('21', 634), ('536', 634), ('fwrite', 633), ('CreatePipe', 630), ('RegisterWindowMessageW', 628), ('CheckMenuItem', 628), ('GetPrivateProfileIntW', 627), ('IntersectRect', 626), ('_errno', 625), ('SetConsoleCtrlHandler', 625), ('FindWindowA', 624), ('strstr', 624), ('GetDateFormatW', 623), ('11', 621), ('RedrawWindow', 618), ('calloc', 617), ('FreeResource', 617), ('InflateRect', 617), ('Polyline', 617), ('FindFirstFileExA', 615), ('309', 612), ('GlobalDeleteAtom', 610), ('_vsnwprintf', 606), ('__vbaStrFixstr', 605), ('GetWindowPlacement', 604), ('GlobalHandle', 599), ('694', 597), ('15', 595), ('303', 595), ('52', 594), ('577', 594), ('__vbaUI1I2', 594), ('CommandLineToArgvW', 592), ('553', 592), ('GetDriveTypeW', 592), ('__vbaGet3', 584), ('QueryPerformanceFrequency', 583), ('Escape', 582), ('IsChild', 580), ('_encoded_null', 578), ('GetMenuItemID', 577), ('public: void __thiscall type_info::_type_info_dtor_internal_method(void)', 577), ('InternetOpenW', 575), ('20', 574), ('RegQueryInfoKeyA', 566), ('SendDlgItemMessageW', 564), ('GetClassNameA', 559), ('GlobalReAlloc', 558), ('__CxxFrameHandler', 557), ('__vbaAryMove', 555), ('FindWindowW', 554), ('SetWindowExtEx', 553), ('300', 553), ('ClosePrinter', 553), ('AlphaBlend', 552), ('VariantCopy', 550), ('__vbaPut3', 549), ('DrawIcon', 548), ('GetTimeFormatW', 546), ('SetViewportExtEx', 545), ('PathFileExistsA', 544), ('SetSecurityDescriptorDacl', 544), ('strcmp', 543), ('ExtTextOutW', 542), ('713', 539), ('GetClassNameW', 538), ('667', 537), ('InitializeSecurityDescriptor', 537), ('wvsprintfW', 536), ('GetComputerNameA', 535), ('696', 534), ('111', 532), ('Ellipse', 530), ('strcpy', 530), ('545', 530), ('FindFirstFileExW', 530), ('GetEnvironmentVariableW', 528), ('__vbaVarSetVar', 527), ('DosDateTimeToFileTime', 526), ('__vbaVarCmpNe', 526), ('GetMenuState', 524), ('521', 521), ('GetWindowTextLengthW', 518), ('GetFileSizeEx', 516), ('PathAppendW', 516), ('LPtoDP', 514), ('SetPropA', 512), ('GetTimeFormatA', 510), ('162', 509), ('569', 506), ('GetUserDefaultUILanguage', 505), ('__vbaCastObj', 504), ('GetOpenFileNameA', 503), ('OpenMutexA', 503), ('GetTopWindow', 502), ('__vbaCopyBytes', 502), ('RegEnumKeyExA', 502), ('IsRectEmpty', 501), ('IsBadCodePtr', 500), ('OpenEventW', 498), ('__vbaLateIdSt', 497), ('WinExec', 495), ('GetPropA', 494), ('PathFindFileNameW', 494), ('__vbaLateIdCall', 493), ('timeGetTime', 492), ('DragFinish', 489), ('GetLengthSid', 489), ('__vbaVarForInit', 488), ('__vbaVarForNext', 488), ('RemovePropA', 485), ('UnregisterClassW', 485), ('CoTaskMemRealloc', 485), ('EqualRect', 484), ('IsDialogMessageA', 483), ('__vbaVarLateMemCallLdRf', 482), ('GetDateFormatA', 480), ('150', 480), ('SetMenu', 479), ('_strnicmp', 479), ('__vbaI4Str', 478), ('GetTextExtentPointA', 476), ('__vbaLateMemSt', 475), ('CoCreateGuid', 474), ('CreateSemaphoreA', 474), ('SetWindowsHookExA', 474), ('ScaleWindowExtEx', 473), ('fopen', 473), ('GetTextMetricsW', 471), ('669', 471), ('__vbaVarCmpGt', 470), ('GlobalFlags', 466), ('IsDialogMessageW', 466), ('BringWindowToTop', 466), ('ScaleViewportExtEx', 465), ('__vbaVar2Vec', 465), ('__vbaAryCopy', 465), ('_iob', 464)]
func_name_o = [('GetProcAddress', 4906), ('GetLastError', 4832), ('GetCurrentProcess', 4481), ('CloseHandle', 4473), ('Sleep', 4362), ('MultiByteToWideChar', 4234), ('GetTickCount', 4104), ('ExitProcess', 4001), ('WriteFile', 3895), ('GetCurrentThreadId', 3791), ('GetModuleHandleA', 3647), ('RegCloseKey', 3631), ('FreeLibrary', 3542), ('UnhandledExceptionFilter', 3435), ('QueryPerformanceCounter', 3434), ('GetCurrentProcessId', 3415), ('WideCharToMultiByte', 3375), ('LoadLibraryA', 3265), ('GetSystemTimeAsFileTime', 3246), ('ReadFile', 3228), ('TerminateProcess', 3227), ('SetFilePointer', 3140), ('SetUnhandledExceptionFilter', 3074), ('DeleteCriticalSection', 3067), ('LeaveCriticalSection', 3012), ('EnterCriticalSection', 3009), ('WaitForSingleObject', 2834), ('GetCommandLineA', 2720), ('GetModuleFileNameA', 2610), ('FindClose', 2559), ('GetFileSize', 2545), ('CreateFileA', 2473), ('RtlUnwind', 2447), ('InitializeCriticalSection', 2413), ('LocalFree', 2399), ('_initterm', 2394), ('GetDC', 2394), ('GetStdHandle', 2394), ('VirtualAlloc', 2373), ('CoTaskMemFree', 2325), ('VirtualFree', 2314), ('CoCreateInstance', 2308), ('lstrlenA', 2308), ('DeleteObject', 2304), ('CreateThread', 2300), ('RaiseException', 2293), ('free', 2281), ('GetWindowRect', 2262), ('TlsGetValue', 2260), ('SelectObject', 2254), ('IsDebuggerPresent', 2242), ('GetClientRect', 2239), ('EnableWindow', 2195), ('GetDeviceCaps', 2134), ('GetFileType', 2132), ('', 2125), ('HeapFree', 2120), ('GetModuleHandleW', 2116), ('ShowWindow', 2082), ('RegQueryValueExA', 2081), ('HeapAlloc', 2073), ('RegOpenKeyExA', 2065), ('TlsSetValue', 2044), ('GetSystemMetrics', 2012), ('SetLastError', 2010), ('GlobalAlloc', 2006), ('DestroyWindow', 2001), ('GetModuleFileNameW', 1991), ('GlobalFree', 1976), ('IsWindow', 1968), ('SetWindowPos', 1964), ('SetTextColor', 1956), ('InvalidateRect', 1943), ('GetDlgItem', 1920), ('GetVersion', 1898), ('SetTimer', 1880), ('malloc', 1871), ('__dllonexit', 1851), ('_onexit', 1838), ('SetBkMode', 1834), ('_amsg_exit', 1830), ('SetBkColor', 1820), ('GlobalLock', 1814), ('GlobalUnlock', 1812), ('EndDialog', 1811), ('GetExitCodeProcess', 1811), ('SetForegroundWindow', 1789), ('GetACP', 1781), ('CreateFileW', 1768), ('SetCursor', 1757), ('memset', 1711), ('PostQuitMessage', 1698), ('EndPaint', 1696), ('BeginPaint', 1696), ('GetSysColor', 1685), ('FillRect', 1682), ('GetProcessHeap', 1664), ('ScreenToClient', 1663), ('HeapReAlloc', 1639), ('MessageBoxA', 1639), ('LocalAlloc', 1631), ('InterlockedExchange', 1626), ('SetEndOfFile', 1623), ('RegOpenKeyExW', 1621), ('SendMessageA', 1620), ('memcpy', 1614), ('_except_handler4_common', 1613), ('wsprintfA', 1607), ('IsWindowVisible', 1591), ('17', 1585), ('SetErrorMode', 1579), ('MulDiv', 1576), ('TlsAlloc', 1562), ('RegQueryValueExW', 1543), ('_lock', 1518), ('_unlock', 1518), ('LoadLibraryW', 1487), ('CharNextA', 1473), ('LoadLibraryExW', 1471), ('GetCPInfo', 1470), ('SetFileTime', 1468), ('TlsFree', 1466), ('lstrcmpiA', 1466), ('GetOEMCP', 1440), ('IsWindowEnabled', 1426), ('OleInitialize', 1424), ('GetStringTypeW', 1419), ('InterlockedDecrement', 1408), ('lstrcpynA', 1408), ('GetEnvironmentStringsW', 1404), ('OleUninitialize', 1404), ('FreeEnvironmentStringsW', 1403), ('LCMapStringW', 1401), ('InterlockedIncrement', 1388), ('HeapSize', 1387), ('GetStartupInfoA', 1383), ('lstrlenW', 1377), ('CreateBrushIndirect', 1363), ('FlushFileBuffers', 1358), ('DeleteFileA', 1358), ('FindFirstFileA', 1348), ('GetCommandLineW', 1343), ('GetMessagePos', 1343), ('GetFileAttributesA', 1323), ('CreatePopupMenu', 1320), ('ImageList_Create', 1317), ('TrackPopupMenu', 1307), ('ImageList_Destroy', 1302), ('lstrcmpA', 1296), ('OpenProcessToken', 1286), ('DecodePointer', 1282), ('CompareFileTime', 1278), ('EnableMenuItem', 1259), ('CloseClipboard', 1257), ('OpenClipboard', 1257), ('VirtualQuery', 1256), ('SHGetSpecialFolderLocation', 1256), ('ExitWindowsEx', 1253), ('DeleteFileW', 1251), ('RegSetValueExA', 1246), ('EmptyClipboard', 1233), ('ShellExecuteA', 1231), ('GetWindowLongA', 1231), ('RegSetValueExW', 1229), ('SetClipboardData', 1228), ('IsProcessorFeaturePresent', 1225), ('InterlockedCompareExchange', 1225), ('EncodePointer', 1216), ('2', 1214), ('InitializeCriticalSectionAndSpinCount', 1210), ('CheckDlgButton', 1210), ('DispatchMessageA', 1203), ('_initterm_e', 1199), ('FindFirstFileW', 1195), ('SetStdHandle', 1192), ('FindNextFileA', 1182), ('CreateDirectoryA', 1181), ('CreateFontIndirectA', 1171), ('ImageList_AddMasked', 1171), ('HeapCreate', 1170), ('RegCreateKeyExW', 1170), ('GetFileAttributesW', 1155), ('LoadCursorA', 1146), ('VirtualProtect', 1145), ('HeapDestroy', 1145), ('SetEvent', 1144), ('9', 1138), ('CreateProcessA', 1133), ('ReleaseDC', 1130), ('GetSystemMenu', 1127), ('RegCreateKeyExA', 1115), ('6', 1114), ('SendMessageW', 1109), ('PeekMessageA', 1107), ('SetWindowLongA', 1103), ('GetTempPathA', 1102), ('DefWindowProcA', 1098), ('GetFullPathNameA', 1096), ('void __cdecl terminate(void)', 1093), ('GetVersionExW', 1091), ('GetVersionExA', 1090), ('LoadLibraryExA', 1088), ('CreateWindowExA', 1086), ('LoadResource', 1077), ('IsValidCodePage', 1071), ('CoUninitialize', 1068), ('memmove', 1061), ('CreateDirectoryW', 1061), ('_crt_debugger_hook', 1053), ('GetPrivateProfileStringA', 1050), ('8', 1049), ('CreateCompatibleDC', 1047), ('RegDeleteKeyA', 1044), ('SetWindowTextA', 1040), ('GetConsoleMode', 1040), ('SetHandleCount', 1031), ('__CxxFrameHandler3', 1030), ('GetWindowsDirectoryA', 1028), ('GetLocaleInfoA', 1025), ('RegDeleteValueA', 1022), ('GetSystemDirectoryA', 1013), ('VerQueryValueA', 1009), ('RegisterClassA', 1008), ('SystemParametersInfoA', 1005), ('lstrcatA', 1000), ('FindNextFileW', 999), ('DeleteDC', 998), ('RegEnumKeyA', 990), ('BitBlt', 987), ('RemoveDirectoryA', 984), ('SetFileAttributesA', 983), ('GetTempPathW', 979), ('GetStockObject', 977), ('_XcptFilter', 974), ('WriteConsoleW', 972), ('_adjust_fdiv', 972), ('SetCurrentDirectoryA', 970), ('CharNextW', 970), ('GetFileVersionInfoA', 967), ('GetSystemInfo', 966), ('GetFileVersionInfoSizeA', 965), ('GetParent', 964), ('RegEnumValueA', 960), ('CreateProcessW', 958), ('lstrcpyA', 950), ('GetConsoleCP', 945), ('GetTempFileNameA', 944), ('GetSystemTime', 942), ('DialogBoxParamA', 941), ('GetStartupInfoW', 940), ('RegDeleteValueW', 936), ('WritePrivateProfileStringA', 932), ('GetWindowLongW', 928), ('CopyFileA', 927), ('SetDlgItemTextA', 925), ('DrawTextA', 925), ('GetDiskFreeSpaceA', 925), ('SetWindowLongW', 924), ('MoveFileA', 923), ('CallWindowProcA', 922), ('SizeofResource', 916), ('SetWindowTextW', 909), ('CoInitialize', 908), ('ShellExecuteW', 908), ('RegDeleteKeyW', 906), ('DisableThreadLibraryCalls', 899), ('CreateWindowExW', 898), ('LockResource', 897), ('LoadCursorW', 895), ('DispatchMessageW', 894), ('DefWindowProcW', 894), ('TranslateMessage', 887), ('LoadBitmapA', 887), ('GetShortPathNameA', 885), ('LoadImageA', 883), ('SHGetPathFromIDListA', 868), ('ExpandEnvironmentStringsA', 862), ('_CxxThrowException', 860), ('__CppXcptFilter', 858), ('lstrcmpiW', 850), ('GetDlgItemTextA', 849), ('GetLocaleInfoW', 848), ('_purecall', 843), ('GetClassInfoA', 842), ('CoTaskMemAlloc', 836), ('LCMapStringA', 835), ('CopyFileW', 834), ('CreateDialogParamA', 828), ('RemoveDirectoryW', 827), ('OpenProcess', 827), ('GetStringTypeA', 826), ('SetFileAttributesW', 826), ('AppendMenuA', 826), ('CompareStringW', 825), ('CreateSolidBrush', 824), ('CharPrevA', 822), ('SHBrowseForFolderA', 816), ('UnmapViewOfFile', 812), ('__clean_type_info_names_internal', 812), ('SearchPathA', 809), ('_malloc_crt', 808), ('PeekMessageW', 794), ('MapViewOfFile', 792), ('GetEnvironmentStrings', 790), ('GetFullPathNameW', 786), ('OutputDebugStringW', 785), ('CreateEventW', 784), ('GetSystemDirectoryW', 781), ('VerQueryValueW', 779), ('ExpandEnvironmentStringsW', 779), ('CreateCompatibleBitmap', 774), ('SysAllocStringLen', 772), ('FindWindowExA', 772), ('FormatMessageW', 764), ('SHGetFileInfoA', 763), ('SetClassLongA', 763), ('exit', 752), ('FreeEnvironmentStringsA', 749), ('MessageBoxW', 748), ('SHFileOperationA', 745), ('GetLocalTime', 743), ('GetTempFileNameW', 740), ('lstrcpynW', 738), ('KillTimer', 737), ('SendMessageTimeoutA', 732), ('4', 728), ('ResetEvent', 727), ('DrawTextW', 727), ('CreateFontIndirectW', 721), ('FindResourceW', 718), ('LoadImageW', 716), ('FormatMessageA', 705), ('lstrcmpW', 702), ('CallWindowProcW', 700), ('InitCommonControls', 698), ('atoi', 698), ('SystemParametersInfoW', 691), ('GetFileVersionInfoW', 688), ('GetFileVersionInfoSizeW', 688), ('7', 684), ('MoveFileW', 681), ('12', 679), ('DialogBoxParamW', 676), ('GetThreadLocale', 673), ('GetWindowsDirectoryW', 661), ('wsprintfW', 659), ('SetEnvironmentVariableA', 658), ('SetCurrentDirectoryW', 652), ('RegEnumValueW', 651), ('GetDesktopWindow', 651), ('LookupPrivilegeValueA', 643), ('__setusermatherr', 636), ('_exit', 632), ('_cexit', 631), ('FileTimeToSystemTime', 628), ('RegisterClassW', 618), ('SetFocus', 618), ('PostMessageW', 617), ('SetDlgItemTextW', 615), ('GetCurrentThread', 604), ('ExitThread', 602), ('GetPrivateProfileStringW', 601), ('UpdateWindow', 590), ('OutputDebugStringA', 588), ('GetTokenInformation', 582), ('MessageBoxIndirectA', 582), ('GetWindow', 579), ('LoadBitmapW', 577), ('RegEnumKeyW', 575), ('SetViewportOrgEx', 574), ('GetUserDefaultLCID', 574), ('_encoded_null', 574), ('strncmp', 573), ('GetEnvironmentVariableA', 573), ('__set_app_type', 571), ('SaveDC', 568), ('WritePrivateProfileStringW', 567), ('CreateEventA', 565), ('LoadStringW', 563), ('SearchPathW', 560), ('ReleaseMutex', 559), ('SHGetPathFromIDListW', 559), ('10', 554), ('_errno', 554), ('lstrcpyW', 554), ('GetTimeZoneInformation', 551), ('GetObjectA', 548), ('StretchBlt', 546), ('public: void __thiscall type_info::_type_info_dtor_internal_method(void)', 545), ('PostMessageA', 544), ('FindWindowExW', 544), ('CreateBitmap', 542), ('void __cdecl operator delete(void *)', 542), ('ResumeThread', 540), ('RegEnumKeyExW', 531), ('VariantClear', 530), ('public: virtual __thiscall type_info::~type_info(void)', 530), ('sprintf', 530), ('strchr', 529), ('CopyRect', 528), ('GetCursorPos', 525), ('calloc', 523), ('VariantChangeTypeEx', 523), ('SysStringLen', 519), ('fwrite', 518), ('VariantCopyInd', 514), ('GetFocus', 513), ('SHGetFileInfoW', 512), ('MoveWindow', 512), ('GetModuleHandleExW', 509), ('OffsetRect', 508), ('SystemTimeToFileTime', 507), ('RestoreDC', 506), ('RectVisible', 506), ('_wcsicmp', 506), ('WaitForMultipleObjects', 505), ('CompareStringA', 505), ('PtInRect', 503), ('realloc', 503), ('GetShortPathNameW', 500), ('CreateDialogParamW', 498), ('GetClassInfoW', 497), ('GetCurrentDirectoryW', 494), ('MapWindowPoints', 493), ('GetWindowThreadProcessId', 490), ('16', 490), ('fclose', 488), ('SHBrowseForFolderW', 488), ('SetFilePointerEx', 486), ('SHGetMalloc', 485), ('CreateFileMappingW', 485), ('GetDiskFreeSpaceW', 484), ('AppendMenuW', 482), ('SendMessageTimeoutW', 482), ('FreeSid', 481), ('FindResourceExW', 480), ('GetDlgItemTextW', 479), ('GetObjectW', 478), ('LoadStringA', 477), ('void * __cdecl operator new(unsigned int)', 476), ('StringFromGUID2', 475), ('RegQueryInfoKeyW', 473), ('CharUpperW', 473), ('23', 472), ('IsValidLocale', 471), ('ClientToScreen', 466), ('SetRect', 463), ('SetClassLongW', 460), ('lstrcatW', 459), ('CreateDIBSection', 456), ('ReleaseCapture', 456), ('GetCurrentDirectoryA', 455), ('19', 454), ('strlen', 452), ('MoveToEx', 452), ('Rectangle', 448), ('LineTo', 447), ('CreateMutexW', 447), ('DuplicateHandle', 445), ('SHFileOperationW', 445), ('_vsnwprintf', 444), ('GetClipBox', 438), ('GetKeyState', 437), ('GetForegroundWindow', 437), ('WriteConsoleA', 433), ('GetConsoleOutputCP', 432), ('CharPrevW', 431), ('__p__commode', 429), ('memcpy_s', 427), ('266', 425), ('ShellExecuteExW', 421), ('GetTextExtentPoint32A', 419), ('15', 419), ('AdjustTokenPrivileges', 417), ('300', 417), ('GetActiveWindow', 416), ('AllocateAndInitializeSid', 413), ('SetCapture', 413), ('162', 412), ('SelectPalette', 411), ('RealizePalette', 411), ('CoInitializeEx', 410), ('CreatePalette', 410), ('PatBlt', 406), ('void __cdecl std::_Xlength_error(char const *)', 405), ('FileTimeToLocalFileTime', 405), ('MessageBoxIndirectW', 404), ('abort', 402), ('InitializeSListHead', 401), ('GetWindowTextW', 401), ('GetTextExtentPoint32W', 399), ('__p__fmode', 395), ('strncpy', 393), ('310', 392), ('GetPixel', 391), ('3', 390), ('316', 389), ('void __cdecl std::_Xout_of_range(char const *)', 387), ('SetStretchBltMode', 386), ('CreateRectRgn', 386), ('memcmp', 386), ('__getmainargs', 385), ('IsIconic', 385), ('_encode_pointer', 384), ('_decode_pointer', 384), ('CLSIDFromString', 382), ('_iob', 380), ('GetAsyncKeyState', 380), ('CLSIDFromProgID', 378), ('CreateStreamOnHGlobal', 376), ('MoveFileExW', 376), ('IsDlgButtonChecked', 375), ('265', 371), ('SetWindowOrgEx', 369), ('InflateRect', 365), ('LoadIconW', 365), ('fprintf', 363), ('HeapSetInformation', 363), ('SHGetFolderPathW', 362), ('115', 360), ('GetDIBits', 359), ('strrchr', 357), ('GetMessageW', 356), ('277', 355), ('CreateDIBitmap', 354), ('CoTaskMemRealloc', 353), ('_stricmp', 352), ('LoadIconA', 351), ('strstr', 350), ('_CIsqrt', 349), ('GetDlgCtrlID', 349), ('InitCommonControlsEx', 348), ('fflush', 347), ('CreatePen', 346), ('IntersectClipRect', 342), ('RegisterWindowMessageW', 340), ('CreateFileMappingA', 340), ('EqualSid', 339), ('GetSubMenu', 339), ('AlphaBlend', 337), ('fopen', 337), ('SetMapMode', 336), ('GetWindowDC', 336), ('wcschr', 335), ('IntersectRect', 334), ('FindResourceA', 333), ('595', 331), ('WaitForSingleObjectEx', 330), ('GetFileTime', 330), ('GetSystemPaletteEntries', 328), ('_except_handler3', 328), ('SendDlgItemMessageA', 324), ('SetThreadPriority', 324), ('_CIcos', 323), ('_CIsin', 323), ('UnregisterClassA', 323), ('CoCreateGuid', 321), ('GetDriveTypeA', 321), ('SetPixel', 321), ('GetTextMetricsA', 321), ('GetPaletteEntries', 321), ('_controlfp_s', 319), ('_configthreadlocale', 319), ('Escape', 318), ('GetFileSizeEx', 316), ('161', 316), ('timeGetTime', 315), ('InternetOpenW', 314), ('_CIlog', 313), ('_allmul', 312), ('ExcludeClipRect', 311), ('InitializeSecurityDescriptor', 311), ('GetCapture', 311), ('SetActiveWindow', 310), ('DestroyMenu', 310), ('_callnewh', 310), ('SetEnvironmentVariableW', 310), ('150', 309), ('GetBitmapBits', 308), ('_CIexp', 308), ('DrawFocusRect', 307), ('ReleaseSemaphore', 306), ('QueryPerformanceFrequency', 304), ('CreateToolhelp32Snapshot', 304), ('SetROP2', 303), ('CallNextHookEx', 303), ('UnhookWindowsHookEx', 303), ('GetMessageA', 301), ('GetUserDefaultUILanguage', 300), ('_calloc_crt', 299), ('EnumWindows', 299), ('__vbaExceptHandler', 297), ('UnregisterClassW', 297), ('CreateMutexA', 297), ('100', 296), ('_CItan', 296), ('__crtTerminateProcess', 296), ('__crtUnhandledException', 296), ('SysFreeString', 295), ('BringWindowToTop', 294), ('SetSecurityDescriptorDacl', 293), ('DestroyIcon', 293), ('111', 293), ('UnrealizeObject', 290), ('_wcsnicmp', 290), ('void __cdecl std::_Xbad_alloc(void)', 289), ('RegEnumKeyExA', 289), ('_CIatan', 289), ('InternetCloseHandle', 289), ('MsgWaitForMultipleObjects', 287), ('MessageBeep', 287), ('CommandLineToArgvW', 287), ('_vsnprintf', 284), ('21', 284), ('EVENT_SINK_AddRef', 283), ('EVENT_SINK_Release', 283), ('EVENT_SINK_QueryInterface', 283), ('311', 282), ('305', 281), ('IsBadReadPtr', 280), ('RegisterClassExW', 280), ('GetWindowTextLengthW', 279), ('WinVerifyTrust', 279), ('void __cdecl operator delete[](void *)', 278), ('GetMenu', 278), ('GetWindowOrgEx', 277), ('SysReAllocStringLen', 277), ('CloseServiceHandle', 277), ('_controlfp', 277), ('wvsprintfW', 276), ('GetWindowTextA', 275), ('FlushInstructionCache', 275), ('LocalFileTimeToFileTime', 273), ('_adj_fptan', 273), ('_adj_fdiv_m64', 273), ('_adj_fprem1', 273), ('_adj_fdiv_m32', 273), ('_adj_fdiv_m16i', 273), ('_adj_fdivr_m16i', 273), ('__vbaChkstk', 273), ('_adj_fpatan', 273), ('_adj_fprem', 273), ('_adj_fdivr_m64', 273), ('__vbaFPException', 273), ('_adj_fdiv_m32i', 273), ('_adj_fdivr_m32i', 273), ('_adj_fdivr_m32', 273), ('_adj_fdiv_r', 273), ('116', 273), ('__vbaHresultCheckObj', 272), ('getenv', 271), ('InternetReadFile', 271), ('fread', 270), ('GetSysColorBrush', 270), ('strcmp', 270), ('GetKeyboardType', 269), ('GlobalHandle', 269), ('CoFreeUnusedLibraries', 269), ('PathFindFileNameW', 268), ('AdjustWindowRectEx', 268), ('RegisterWindowMessageA', 267), ('__vbaFreeObj', 267), ('LookupPrivilegeValueW', 267), ('18', 267), ('__vbaFreeStr', 267), ('LPtoDP', 265), ('CheckMenuItem', 265), ('GetLastActivePopup', 264), ('__vbaObjSet', 263), ('public: __thiscall std::exception::exception(class std::exception const &)', 263), ('vfprintf', 262), ('__vbaFreeVarList', 262), ('IsChild', 262), ('wcsstr', 262), ('wcslen', 262), ('__vbaFreeVar', 262), ('DeviceIoControl', 261), ('149', 261), ('CryptReleaseContext', 261), ('GetMenuItemCount', 260), ('CreatePenIndirect', 258), ('printf', 257), ('GetClassInfoExW', 257), ('__vbaStrMove', 257), ('TextOutA', 256), ('RedrawWindow', 256), ('Ellipse', 255), ('20', 255), ('307', 255), ('GetCurrentPositionEx', 254), ('TerminateThread', 253), ('537', 253), ('Process32NextW', 253), ('OpenThreadToken', 252), ('public: virtual __thiscall std::exception::~exception(void)', 252), ('SetConsoleCtrlHandler', 252), ('RegQueryInfoKeyA', 251), ('GetWindowPlacement', 251), ('163', 251), ('Process32FirstW', 251), ('__vbaNew2', 251), ('__vbaFreeStrList', 250), ('StringFromCLSID', 249), ('qsort', 249), ('FreeResource', 248), ('ExtTextOutA', 247), ('_invoke_watson', 246), ('800', 246), ('DllFunctionCall', 246), ('EqualRect', 245), ('__vbaStrCat', 244), ('Polyline', 242), ('__vbaVarDup', 242), ('_time64', 242), ('__vbaStrCopy', 242), ('ExtTextOutW', 241), ('11', 241), ('_wtoi', 241), ('__vbaFreeObjList', 240), ('24', 240), ('public: virtual char const * __thiscall std::exception::what(void)const ', 236), ('wcsrchr', 236), ('SetWindowExtEx', 235), ('PathRemoveFileSpecW', 235), ('__vbaOnError', 234), ('public: __thiscall std::exception::exception(char const * const &)', 232), ('22', 232), ('atol', 232), ('ReadConsoleW', 232), ('__vbaObjSetAddref', 232), ('ClosePrinter', 231), ('SendDlgItemMessageW', 231), ('__vbaStrVarMove', 231), ('GetMenuItemID', 230), ('GetLengthSid', 228), ('SetDIBColorTable', 228), ('SetBrushOrgEx', 228), ('GlobalReAlloc', 228), ('200', 228), ('wcsncmp', 227), ('SHGetSpecialFolderPathW', 226), ('SetViewportExtEx', 225), ('isdigit', 225), ('RegOpenKeyA', 224), ('__CxxFrameHandler', 224), ('StretchDIBits', 223), ('CreateEnvironmentBlock', 223), ('GetUserDefaultLangID', 223), ('IsBadWritePtr', 223), ('685', 223), ('GdipFree', 223), ('__vbaVarMove', 223), ('UnlockFile', 222), ('GetMonitorInfoW', 222), ('__vbaStrCmp', 222), ('LockFile', 221), ('LocalReAlloc', 221), ('CreateRectRgnIndirect', 221), ('IsDBCSLeadByte', 221), ('PathFileExistsW', 220), ('IsRectEmpty', 220), ('GetClassNameW', 220), ('EnumSystemLocalesW', 220), ('1', 220), ('GdiplusStartup', 219), ('__vbaStrVarVal', 219), ('__vbaErrorOverflow', 219), ('strerror', 218), ('MonitorFromWindow', 218), ('tolower', 218), ('fseek', 217), ('GetTextMetricsW', 217), ('atof', 217), ('OpenPrinterA', 216), ('GetDriveTypeW', 216), ('isspace', 216), ('GetEnvironmentVariableW', 216), ('__vbaSetSystemError', 216), ('DrawIcon', 215), ('EnumSystemLocalesA', 214), ('_mbscmp', 213), ('SelectClipRgn', 212), ('fputc', 212), ('__vbaLenBstr', 212), ('GetMenuState', 211), ('GetTopWindow', 211), ('InternetCrackUrlW', 211), ('RoundRect', 210), ('_strnicmp', 210), ('SetWindowRgn', 209), ('FindFirstFileExW', 209), ('__vbaExitProc', 208), ('_close', 207), ('CreateHalftonePalette', 207), ('WindowFromPoint', 207), ('_TrackMouseEvent', 207), ('OpenEventW', 207), ('GdipDisposeImage', 207), ('52', 207), ('GetDateFormatA', 206), ('HttpQueryInfoW', 206), ('__vbaVarCat', 206), ('__vbaStrToAnsi', 206), ('PathAppendW', 205), ('__vbaI4Var', 205), ('memchr', 204), ('SetRectEmpty', 204), ('memmove_s', 204), ('PathFindExtensionW', 204), ('char const * __cdecl std::_Syserror_map(int)', 203), ('GetOpenFileNameA', 203), ('floor', 203), ('GdiplusShutdown', 203), ('FindWindowW', 202), ('GetDateFormatW', 201), ('GlobalDeleteAtom', 201), ('_strdup', 200), ('SetMenu', 200), ('ValidateRect', 200), ('ceil', 200), ('GetRgnBox', 199), ('2614', 199), ('__vbaVarTstEq', 199), ('sscanf', 199), ('OpenEventA', 198), ('char const * __cdecl std::_Winerror_map(int)', 198), ('GetUserNameW', 198), ('CryptCreateHash', 198), ('__iob_func', 197), ('strtol', 197), ('_acmdln', 196), ('GetClassNameA', 196), ('616', 196), ('CryptDestroyHash', 196), ('GetDIBColorTable', 195), ('wcscpy_s', 195), ('GetTextExtentPointA', 194), ('201', 194), ('RegOpenKeyW', 194), ('CryptHashData', 194), ('__vbaStrToUnicode', 193), ('VerSetConditionMask', 192), ('FindWindowA', 192), ('GetNativeSystemInfo', 192), ('InterlockedPushEntrySList', 192), ('CryptAcquireContextW', 192), ('184', 191), ('__vbaStrI4', 191), ('ImageList_Add', 190), ('ImageList_ReplaceIcon', 189), ('ShellExecuteExA', 188), ('CoGetClassObject', 188), ('617', 188), ('__wgetmainargs', 188), ('__vbaI2I4', 187), ('CombineRgn', 186), ('5139', 186), ('IsDialogMessageW', 186), ('GetProcessId', 186), ('GetFileAttributesExW', 185), ('CoRevokeClassObject', 184), ('ImageList_DragMove', 184), ('ImageList_DragEnter', 184), ('ImageList_EndDrag', 184), ('ImageList_BeginDrag', 184), ('GetMessageTime', 184), ('_fmode', 183), ('__vbaCastObj', 183), ('ImageList_SetBkColor', 182), ('GlobalSize', 182), ('GetPrivateProfileIntA', 182), ('581', 182), ('GetBkColor', 181), ('ImageList_DragLeave', 181), ('GetTimeFormatA', 181), ('IsBadCodePtr', 181), ('DosDateTimeToFileTime', 180), ('IsEqualGUID', 180), ('OpenSCManagerW', 180), ('185', 180), ('5', 180), ('strtoul', 180), ('Arc', 179), ('PtVisible', 179), ('_write', 179), ('Pie', 178), ('OpenServiceW', 178), ('__vbaStrI2', 178), ('__vbaAryDestruct', 178), ('CreateProcessAsUserW', 177), ('InternetSetOptionW', 177), ('_read', 177), ('__vbaInStr', 177), ('ftell', 176), ('__vbaGenerateBoundsError', 176), ('GetTimeFormatW', 175), ('rand', 175), ('321', 175), ('GetExitCodeThread', 175), ('__vbaVarCopy', 175), ('GetDCOrgEx', 174), ('PostThreadMessageW', 174), ('ImageList_DragShowNolock', 173), ('ImageList_Draw', 173), ('GetNextDlgTabItem', 173), ('OffsetViewportOrgEx', 173), ('__vbaBoolVarNull', 173), ('SHDeleteKeyW', 173), ('_beginthreadex', 172), ('SetParent', 172), ('_snprintf', 172), ('SuspendThread', 170), ('484', 170), ('632', 170), ('608', 170), ('GetSaveFileNameA', 169), ('ScaleViewportExtEx', 169), ('CreateFontA', 169), ('CreatePatternBrush', 169), ('GetDiskFreeSpaceExW', 169), ('ImageList_GetIconSize', 168), ('IsDialogMessageA', 168), ('520', 168), ('ScaleWindowExtEx', 167), ('toupper', 167), ('RegCreateKeyA', 166), ('601', 166), ('4889', 165), ('600', 165), ('AttachThreadInput', 165), ('CharUpperA', 164), ('SetWindowsHookExW', 164), ('ExtFloodFill', 163), ('112', 163), ('1137', 163), ('CreateFontW', 163), ('14059', 162), ('TextOutW', 162), ('3346', 162), ('561', 162), ('strcpy', 162), ('__vbaVarAdd', 162), ('ControlService', 161), ('ShowScrollBar', 161), ('GetBrushOrgEx', 160), ('LoadMenuW', 160), ('540', 160), ('GlobalFlags', 159), ('4424', 159), ('13', 159), ('__vbaI2Var', 159), ('fputs', 159), ('GetFileTitleA', 158), ('MaskBlt', 158), ('631', 158), ('3663', 158), ('bsearch', 158), ('186', 158), ('SetTextAlign', 157), ('ImageList_GetImageCount', 157), ('485', 157), ('2339', 157), ('4589', 157), ('__vbaRedim', 157), ('CreateDCA', 156), ('SetMenuItemBitmaps', 156), ('598', 156), ('645', 156), ('InterlockedPopEntrySList', 156), ('77', 156), ('712', 156), ('14', 155), ('SetWindowsHookExA', 155), ('MoveFileExA', 155), ('QueryServiceStatus', 155), ('InitializeAcl', 155), ('__vbaVarTstNe', 155), ('fgets', 155), ('IsDBCSLeadByteEx', 154), ('SetScrollPos', 154), ('GetPropW', 154), ('SetPropW', 154), ('ReleaseSRWLockExclusive', 153), ('DeregisterEventSource', 153), ('VerifyVersionInfoW', 153), ('GetMenuCheckMarkDimensions', 153), ('GetTextColor', 153), ('CertCloseStore', 153), ('CryptMsgClose', 153), ('__vbaAryUnlock', 153), ('4425', 152), ('618', 152), ('CreateWellKnownSid', 152), ('InternetOpenUrlW', 152), ('286', 152), ('__vbaVarSub', 152), ('__vbaFpI4', 152), ('AcquireSRWLockExclusive', 151), ('ImageList_GetDragImage', 151), ('__vbaEnd', 151)]
func_name_x = [('', 159559), ('GetProcAddress', 11255), ('ExitProcess', 11220), ('GetLastError', 8838), ('WriteFile', 8662), ('CloseHandle', 8640), ('GetModuleHandleA', 8606), ('MultiByteToWideChar', 8014), ('GetCommandLineA', 7555), ('LoadLibraryA', 7515), ('FreeLibrary', 7386), ('RegCloseKey', 7377), ('GetModuleFileNameA', 7285), ('GetTickCount', 6928), ('Sleep', 6814), ('ReadFile', 6730), ('GetCurrentProcess', 6566), ('FindClose', 6486), ('SetFilePointer', 6341), ('VirtualAlloc', 6311), ('lstrlenA', 6113), ('WideCharToMultiByte', 5948), ('CreateFileA', 5878), ('GetCurrentThreadId', 5800), ('GetStdHandle', 5623), ('RtlUnwind', 5551), ('DeleteObject', 5544), ('RegQueryValueExA', 5448), ('GetFileSize', 5408), ('UnhandledExceptionFilter', 5383), ('VirtualFree', 5360), ('RegOpenKeyExA', 5358), ('SelectObject', 5280), ('GetDeviceCaps', 5019), ('CreateThread', 4978), ('SetTextColor', 4967), ('WaitForSingleObject', 4892), ('CharNextA', 4890), ('EnterCriticalSection', 4884), ('SetBkColor', 4879), ('RaiseException', 4878), ('LeaveCriticalSection', 4849), ('FindFirstFileA', 4766), ('SetBkMode', 4735), ('DestroyWindow', 4552), ('GetStartupInfoA', 4496), ('GetFileType', 4437), ('lstrcpynA', 4427), ('DeleteCriticalSection', 4412), ('ImageList_Create', 4411), ('ImageList_Destroy', 4355), ('GetVersion', 4327), ('CreateBrushIndirect', 4304), ('ShellExecuteA', 4189), ('GetDC', 4184), ('LoadLibraryExA', 4131), ('GetACP', 4116), ('CoTaskMemFree', 4106), ('MessageBoxA', 4105), ('ShowWindow', 4041), ('QueryPerformanceCounter', 4033), ('HeapAlloc', 4015), ('CreateFontIndirectA', 3932), ('wsprintfA', 3835), ('VerQueryValueA', 3826), ('InterlockedDecrement', 3818), ('GlobalAlloc', 3817), ('PostQuitMessage', 3773), ('CoCreateInstance', 3763), ('HeapFree', 3682), ('EnableWindow', 3667), ('InterlockedIncrement', 3650), ('GetSystemMetrics', 3612), ('GetCurrentProcessId', 3595), ('GetModuleHandleW', 3573), ('GetWindowRect', 3568), ('GetDlgItem', 3493), ('GetClientRect', 3455), ('TerminateProcess', 3455), ('SetWindowPos', 3406), ('SetForegroundWindow', 3405), ('BeginPaint', 3402), ('GlobalFree', 3392), ('GetFileVersionInfoA', 3381), ('GetFileVersionInfoSizeA', 3376), ('IsWindowVisible', 3370), ('lstrcmpiA', 3359), ('EndPaint', 3313), ('HeapReAlloc', 3285), ('GetSysColor', 3279), ('SetUnhandledExceptionFilter', 3278), ('FindNextFileA', 3272), ('IsWindow', 3270), ('SetErrorMode', 3241), ('DispatchMessageA', 3241), ('EndDialog', 3234), ('GlobalUnlock', 3206), ('GetSystemTimeAsFileTime', 3206), ('IsWindowEnabled', 3200), ('GlobalLock', 3190), ('SetTimer', 3160), ('OleInitialize', 3157), ('TlsGetValue', 3139), ('InitializeCriticalSection', 3114), ('GetLocaleInfoA', 3107), ('CreateProcessA', 3090), ('GetExitCodeProcess', 3079), ('SetLastError', 3074), ('lstrcmpA', 3066), ('DeleteFileA', 3063), ('MulDiv', 3056), ('SendMessageA', 3052), ('DefWindowProcA', 3050), ('100', 3042), ('InvalidateRect', 3040), ('VirtualProtect', 3028), ('GetStringTypeW', 3010), ('CreateDirectoryA', 3001), ('GetModuleFileNameW', 3001), ('__vbaExceptHandler', 3001), ('GetCPInfo', 2994), ('SetFileTime', 2987), ('FillRect', 2986), ('CreateWindowExA', 2977), ('OleUninitialize', 2971), ('SetCursor', 2965), ('SHGetFileInfoA', 2964), ('GetFileAttributesA', 2959), ('TlsSetValue', 2957), ('RegSetValueExA', 2941), ('GetOEMCP', 2940), ('ScreenToClient', 2922), ('CreateFileW', 2922), ('SHFileOperationA', 2918), ('EnableMenuItem', 2904), ('SetFileAttributesA', 2874), ('EVENT_SINK_QueryInterface', 2871), ('EVENT_SINK_Release', 2870), ('LCMapStringW', 2869), ('EVENT_SINK_AddRef', 2869), ('LocalAlloc', 2862), ('LoadCursorA', 2858), ('CreateCompatibleDC', 2855), ('SetEndOfFile', 2852), ('17', 2836), ('lstrcatA', 2826), ('CreatePopupMenu', 2824), ('GetEnvironmentStringsW', 2814), ('GetMessagePos', 2811), ('GetTempPathA', 2805), ('TlsAlloc', 2805), ('LocalFree', 2791), ('FreeEnvironmentStringsW', 2789), ('ExitWindowsEx', 2780), ('DllFunctionCall', 2770), ('GetCommandLineW', 2762), ('GetStockObject', 2758), ('OpenClipboard', 2748), ('CloseClipboard', 2725), ('SHGetSpecialFolderLocation', 2722), ('GetSystemDirectoryA', 2718), ('TrackPopupMenu', 2718), ('GetSystemMenu', 2717), ('VirtualQuery', 2703), ('DeleteDC', 2700), ('EmptyClipboard', 2699), ('TlsFree', 2680), ('RegCreateKeyExA', 2664), ('GetProcessHeap', 2660), ('SetClipboardData', 2645), ('CheckDlgButton', 2640), ('CopyFileA', 2638), ('HeapSize', 2622), ('CompareFileTime', 2605), ('FlushFileBuffers', 2596), ('PeekMessageA', 2586), ('CreateCompatibleBitmap', 2585), ('SysFreeString', 2557), ('SaveDC', 2553), ('GetWindowLongA', 2525), ('SetWindowLongA', 2512), ('ImageList_AddMasked', 2509), ('RegisterClassA', 2495), ('LoadStringA', 2456), ('BitBlt', 2455), ('GetWindowsDirectoryA', 2455), ('CreateSolidBrush', 2417), ('IsValidCodePage', 2406), ('WritePrivateProfileStringA', 2405), ('RemoveDirectoryA', 2404), ('lstrcpyA', 2393), ('SetStdHandle', 2388), ('CallWindowProcA', 2362), ('GetPrivateProfileStringA', 2346), ('GetThreadLocale', 2344), ('SystemParametersInfoA', 2344), ('MoveFileA', 2342), ('IsDebuggerPresent', 2324), ('RegDeleteValueA', 2308), ('SetWindowTextA', 2306), ('ExpandEnvironmentStringsA', 2302), ('DrawTextA', 2283), ('LoadBitmapA', 2277), ('DialogBoxParamA', 2270), ('SetCurrentDirectoryA', 2266), ('InitializeCriticalSectionAndSpinCount', 2264), ('CreateBitmap', 2263), ('HeapCreate', 2255), ('GetTempFileNameA', 2250), ('RegDeleteKeyA', 2238), ('ImageList_Add', 2233), ('GetKeyboardType', 2230), ('AppendMenuA', 2223), ('GetShortPathNameA', 2197), ('GetClassInfoA', 2179), ('StretchBlt', 2174), ('CreateDialogParamA', 2134), ('SetDlgItemTextA', 2133), ('SysAllocStringLen', 2122), ('SearchPathA', 2116), ('GetDiskFreeSpaceA', 2116), ('RestoreDC', 2113), ('FindWindowExA', 2113), ('LoadImageA', 2111), ('GetConsoleMode', 2106), ('SetViewportOrgEx', 2102), ('CharPrevA', 2095), ('GetDlgItemTextA', 2090), ('GetDIBits', 2086), ('CreateDIBSection', 2085), ('WriteConsoleW', 2084), ('GetFullPathNameA', 2078), ('SHGetPathFromIDListA', 2070), ('SendMessageTimeoutA', 2064), ('GetConsoleCP', 2059), ('GetObjectA', 2055), ('LoadLibraryExW', 2054), ('RegEnumKeyA', 2050), ('RegEnumValueA', 2046), ('_CIsin', 2046), ('_CIcos', 2041), ('SetClassLongA', 2036), ('lstrlenW', 2036), ('RealizePalette', 2033), ('_allmul', 2029), ('GetPixel', 2022), ('GetStartupInfoW', 2021), ('MoveToEx', 2020), ('RectVisible', 2018), ('SysReAllocStringLen', 2016), ('_CIsqrt', 2014), ('_CIlog', 2014), ('_CIatan', 2013), ('_CItan', 2012), ('_CIexp', 2012), ('_adj_fdivr_m32i', 2010), ('_adj_fdiv_m16i', 2009), ('_adj_fprem', 2009), ('_adj_fdiv_m32i', 2009), ('_adj_fdiv_m64', 2008), ('_adj_fprem1', 2008), ('_adj_fdivr_m16i', 2008), ('__vbaChkstk', 2008), ('_adj_fdivr_m64', 2008), ('__vbaFPException', 2008), ('_adj_fdivr_m32', 2008), ('_adj_fdiv_r', 2008), ('_adj_fptan', 2007), ('_adj_fdiv_m32', 2007), ('_adj_fpatan', 2007), ('ExcludeClipRect', 1999), ('SHBrowseForFolderA', 1995), ('IntersectClipRect', 1982), ('SelectPalette', 1970), ('ReleaseDC', 1966), ('GetClipBox', 1963), ('__vbaFreeVar', 1950), ('LineTo', 1947), ('__vbaHresultCheckObj', 1947), ('MessageBoxIndirectA', 1933), ('SetWindowOrgEx', 1928), ('CreatePalette', 1920), ('__vbaFreeStr', 1918), ('SetPixel', 1914), ('PatBlt', 1912), ('GetTextMetricsA', 1907), ('SetROP2', 1898), ('CreateDIBitmap', 1887), ('SetStretchBltMode', 1884), ('GetSystemPaletteEntries', 1878), ('ImageList_EndDrag', 1876), ('UnrealizeObject', 1874), ('ImageList_DragMove', 1874), ('ImageList_DragEnter', 1873), ('LoadLibraryW', 1870), ('__vbaStrMove', 1866), ('GetBitmapBits', 1859), ('GetPaletteEntries', 1853), ('__vbaFreeVarList', 1846), ('__vbaNew2', 1842), ('SetBrushOrgEx', 1838), ('GetWindowOrgEx', 1838), ('ImageList_DragLeave', 1827), ('ImageList_BeginDrag', 1823), ('608', 1823), ('GetCurrentPositionEx', 1820), ('GetDIBColorTable', 1816), ('ImageList_SetBkColor', 1809), ('CreatePenIndirect', 1804), ('RegQueryValueExW', 1797), ('CreateHalftonePalette', 1796), ('__vbaFreeObj', 1795), ('GetTextExtentPoint32A', 1793), ('SetDIBColorTable', 1791), ('RegOpenKeyExW', 1786), ('ImageList_DragShowNolock', 1780), ('ImageList_Remove', 1775), ('__vbaSetSystemError', 1773), ('ImageList_SetIconSize', 1763), ('ImageList_GetImageCount', 1763), ('632', 1763), ('GetBrushOrgEx', 1756), ('SetHandleCount', 1756), ('ImageList_GetIconSize', 1755), ('ImageList_DrawEx', 1753), ('MaskBlt', 1751), ('ImageList_Draw', 1738), ('617', 1736), ('ImageList_Read', 1725), ('ImageList_GetBkColor', 1719), ('IsProcessorFeaturePresent', 1703), ('GetDCOrgEx', 1701), ('GetVersionExA', 1695), ('ImageList_Write', 1688), ('616', 1680), ('598', 1675), ('__vbaVarMove', 1654), ('DecodePointer', 1628), ('537', 1626), ('631', 1610), ('TranslateMessage', 1598), ('516', 1581), ('HeapDestroy', 1574), ('__vbaFreeStrList', 1564), ('__vbaStrVarMove', 1559), ('ExitThread', 1543), ('EncodePointer', 1526), ('600', 1516), ('__vbaStrCopy', 1506), ('526', 1472), ('595', 1462), ('__vbaOnError', 1459), ('CompareStringA', 1446), ('__vbaStrCat', 1443), ('FindNextFileW', 1441), ('__vbaVarDup', 1438), ('SHGetFolderPathA', 1436), ('607', 1428), ('525', 1426), ('717', 1421), ('FindFirstFileW', 1418), ('645', 1418), ('619', 1418), ('InternetCloseHandle', 1416), ('send', 1413), ('CoInitialize', 1413), ('529', 1413), ('SetEnvironmentVariableA', 1410), ('644', 1403), ('__vbaI4Var', 1395), ('DeleteFileW', 1388), ('__vbaObjSet', 1385), ('__vbaStrCmp', 1377), ('InterlockedExchange', 1349), ('CoUninitialize', 1329), ('SendMessageW', 1320), ('GetFileAttributesW', 1309), ('685', 1309), ('GetStringTypeA', 1304), ('520', 1303), ('LCMapStringA', 1296), ('593', 1277), ('OpenProcessToken', 1270), ('ShellExecuteExA', 1268), ('DefWindowProcW', 1266), ('RegSetValueExW', 1263), ('GetLocaleInfoW', 1262), ('CompareStringW', 1262), ('518', 1261), ('SetEvent', 1258), ('CreateWindowExW', 1254), ('528', 1254), ('LoadCursorW', 1250), ('InternetOpenA', 1245), ('LoadResource', 1245), ('GetTempPathW', 1245), ('__vbaStrToAnsi', 1218), ('716', 1217), ('__vbaLenBstr', 1217), ('DispatchMessageW', 1207), ('SetFilePointerEx', 1207), ('581', 1204), ('DeleteEnhMetaFile', 1198), ('GetWinMetaFileBits', 1196), ('InternetReadFile', 1192), ('594', 1192), ('CreateDirectoryW', 1188), ('__vbaObjSetAddref', 1185), ('PlayEnhMetaFile', 1182), ('RegCreateKeyExW', 1177), ('570', 1176), ('WSACleanup', 1174), ('WSAStartup', 1174), ('GetModuleHandleExW', 1163), ('URLDownloadToFileA', 1161), ('GetEnhMetaFileHeader', 1160), ('__vbaAryDestruct', 1159), ('__vbaFreeObjList', 1155), ('SetWinMetaFileBits', 1154), ('GetEnhMetaFilePaletteEntries', 1153), ('UpdateWindow', 1153), ('htons', 1152), ('WSAIoctl', 1150), ('SetEnhMetaFileBits', 1146), ('GetEnhMetaFileBits', 1146), ('socket', 1145), ('LoadIconA', 1142), ('connect', 1141), ('GetEnvironmentStrings', 1140), ('FreeEnvironmentStringsA', 1136), ('inet_addr', 1131), ('618', 1130), ('closesocket', 1129), ('CreateProcessW', 1129), ('SetWindowLongW', 1128), ('CopyEnhMetaFileA', 1127), ('__vbaStrVarVal', 1126), ('__vbaStrToUnicode', 1124), ('__vbaFpI4', 1122), ('__vbaVarTstEq', 1118), ('__vbaErrorOverflow', 1116), ('wsprintfW', 1115), ('648', 1113), ('__vbaEnd', 1112), ('GetMessageA', 1109), ('recv', 1108), ('__vbaVarCat', 1106), ('GdipFree', 1104), ('GetWindowLongW', 1103), ('GetLocalTime', 1099), ('OpenProcess', 1097), ('__vbaVarAdd', 1095), ('gethostbyname', 1094), ('GetSystemInfo', 1093), ('GetFullPathNameW', 1092), ('__vbaFileOpen', 1090), ('Netbios', 1088), ('ExtTextOutA', 1086), ('SizeofResource', 1086), ('LockResource', 1084), ('__vbaFileClose', 1084), ('546', 1083), ('711', 1081), ('712', 1077), ('lstrcpynW', 1076), ('GetCurrentThread', 1074), ('606', 1072), ('GetRgnBox', 1070), ('PeekMessageW', 1069), ('lstrcmpW', 1066), ('SetFileAttributesW', 1066), ('ShellExecuteW', 1061), ('waveInOpen', 1058), ('GetUserDefaultLCID', 1058), ('InternetConnectA', 1052), ('acmStreamSize', 1041), ('capGetDriverDescriptionA', 1039), ('FtpPutFileA', 1037), ('CreateEventW', 1035), ('lstrcpyW', 1035), ('inet_ntoa', 1030), ('SetFocus', 1029), ('lstrcmpiW', 1024), ('InternetOpenUrlA', 1019), ('DragQueryFileA', 1018), ('StringFromCLSID', 1012), ('exit', 1007), ('SetWindowTextW', 1003), ('ntohs', 1002), ('CharNextW', 1000), ('GdiFlush', 998), ('IsValidLocale', 986), ('__vbaInStr', 984), ('MethCallEngine', 967), ('GetVersionExW', 959), ('WSAGetLastError', 953), ('_TrackMouseEvent', 949), ('ImageList_ReplaceIcon', 946), ('bind', 943), ('__vbaAryUnlock', 938), ('MoveFileW', 937), ('CreateDCA', 935), ('__vbaRedim', 934), ('ProcCallEngine', 929), ('576', 926), ('GetForegroundWindow', 925), ('FileTimeToSystemTime', 919), ('GetPrivateProfileStringW', 918), ('accept', 914), ('535', 914), ('listen', 913), ('RegDeleteValueW', 913), ('DialogBoxParamW', 913), ('ImageList_GetDragImage', 907), ('GetSystemTime', 903), ('acmStreamOpen', 902), ('CreateFontIndirectW', 902), ('ExpandEnvironmentStringsW', 901), ('573', 901), ('681', 901), ('_initterm', 900), ('SetCurrentDirectoryW', 900), ('GetSystemDirectoryW', 900), ('CopyFileW', 898), ('519', 897), ('__set_app_type', 888), ('ioctlsocket', 885), ('gethostname', 883), ('GdiplusStartup', 881), ('select', 876), ('GdipDisposeImage', 875), ('561', 874), ('__vbaI2Var', 871), ('__vbaExitProc', 870), ('GetTempFileNameW', 865), ('578', 861), ('__p__fmode', 856), ('mciSendStringA', 855), ('RegDeleteKeyW', 854), ('571', 854), ('__vbaVarTstNe', 854), ('CallWindowProcW', 853), ('GdipDeleteGraphics', 851), ('ImageList_SetDragCursorImage', 851), ('GdipAlloc', 849), ('_exit', 848), ('SystemParametersInfoW', 846), ('lstrcatW', 846), ('RemoveDirectoryW', 846), ('getservbyname', 845), ('sendto', 845), ('__vbaAryConstruct2', 843), ('_XcptFilter', 841), ('getsockname', 840), ('GetWindowsDirectoryW', 840), ('__setusermatherr', 838), ('PlaySoundA', 837), ('9', 837), ('acmStreamConvert', 836), ('SetDlgItemTextW', 836), ('GdiplusShutdown', 835), ('__p__commode', 835), ('MessageBoxW', 832), ('__vbaVarSub', 831), ('shutdown', 830), ('LoadBitmapW', 830), ('GdipGetImageEncoders', 829), ('GdipGetImageEncodersSize', 829), ('RegisterClassW', 827), ('610', 824), ('GdipGetImageGraphicsContext', 819), ('GdipCreateBitmapFromScan0', 818), ('gethostbyaddr', 814), ('GdipSaveImageToStream', 812), ('DrawTextW', 812), ('waveInAddBuffer', 811), ('waveInUnprepareHeader', 810), ('waveInClose', 809), ('SetEnvironmentVariableW', 808), ('waveInStart', 806), ('waveInReset', 806), ('waveInPrepareHeader', 806), ('_except_handler3', 806), ('GdipGetImagePixelFormat', 804), ('GdipSetInterpolationMode', 803), ('GetParent', 800), ('acmStreamClose', 799), ('_adjust_fdiv', 799), ('FindWindowExW', 796), ('GdipDrawImageRectI', 793), ('acmStreamUnprepareHeader', 793), ('__vbaGenerateBoundsError', 793), ('acmStreamPrepareHeader', 792), ('AdjustTokenPrivileges', 792), ('acmStreamReset', 790), ('_controlfp', 790), ('612', 790), ('580', 789), ('WritePrivateProfileStringW', 788), ('GetConsoleOutputCP', 788), ('666', 785), ('__vbaObjVar', 785), ('FindResourceW', 782), ('599', 777), ('ResetEvent', 777), ('SHGetPathFromIDListW', 775), ('6', 773), ('UnmapViewOfFile', 772), ('GetDlgItemTextW', 770), ('FindResourceA', 770), ('LoadImageW', 769), ('RegEnumValueW', 765), ('GetDesktopWindow', 762), ('SHGetFileInfoW', 758), ('__vbaBoolVarNull', 757), ('GetShortPathNameW', 755), ('563', 755), ('ReleaseMutex', 755), ('LoadStringW', 747), ('GetWindowThreadProcessId', 742), ('__vbaStrVarCopy', 741), ('660', 739), ('SHBrowseForFolderW', 734), ('GetWindow', 733), ('SHFileOperationW', 731), ('613', 731), ('__WSAFDIsSet', 725), ('EVENT_SINK_GetIDsOfNames', 723), ('319', 721), ('GdipCreateBitmapFromHBITMAP', 719), ('HttpQueryInfoA', 718), ('EVENT_SINK_Invoke', 718), ('Zombie_GetTypeInfo', 718), ('Zombie_GetTypeInfoCount', 718), ('OutputDebugStringW', 714), ('321', 713), ('320', 712), ('NtQuerySystemInformation', 711), ('Rectangle', 708), ('VerQueryValueW', 708), ('AppendMenuW', 706), ('IsEqualGUID', 703), ('__vbaLsetFixstr', 701), ('__vbaInStrVar', 700), ('579', 697), ('__vbaLateMemCall', 697), ('PostMessageA', 697), ('WriteConsoleA', 695), ('InitCommonControls', 694), ('__getmainargs', 692), ('__vbaVarCopy', 692), ('531', 690), ('MapViewOfFile', 690), ('RegEnumKeyW', 689), ('InitializeSListHead', 687), ('714', 685), ('GetClassInfoW', 683), ('__vbaUbound', 682), ('__vbaVarTstGt', 680), ('PostMessageW', 678), ('4', 677), ('__vbaRedimPreserve', 674), ('SendMessageTimeoutW', 669), ('GetFileVersionInfoW', 668), ('__vbaRecAnsiToUni', 665), ('__vbaRecUniToAnsi', 665), ('GetDiskFreeSpaceW', 663), ('626', 663), ('601', 663), ('GetFocus', 662), ('GetFileVersionInfoSizeW', 662), ('__vbaI2I4', 654), ('GetCurrentDirectoryW', 648), ('GetTimeZoneInformation', 645), ('527', 641), ('CreateDialogParamW', 639), ('GetWindowTextA', 635), ('689', 635), ('CreateMutexA', 635), ('DuplicateHandle', 634), ('__vbaPrintFile', 631), ('690', 630), ('2', 630), ('CharPrevW', 629), ('558', 629), ('LoadIconW', 625), ('memset', 623), ('__vbaVarCmpEq', 619), ('710', 615), ('RegisterClassExA', 614), ('SetClassLongW', 614), ('SearchPathW', 610), ('KillTimer', 610), ('GetCursorPos', 609), ('__vbaLenVar', 605), ('115', 603), ('__vbaFpI2', 603), ('592', 601), ('709', 601), ('8', 600), ('ReadConsoleW', 600), ('GetObjectW', 594), ('694', 594), ('__vbaAryLock', 589), ('GetMessageW', 585), ('SetThreadPriority', 584), ('__vbaVarLateMemCallLd', 584), ('memcpy', 582), ('_acmdln', 581), ('__vbaFixstrConstruct', 579), ('572', 578), ('__vbaLateMemCallLd', 577), ('536', 574), ('__vbaVarAnd', 573), ('GetEnvironmentVariableA', 566), ('CreateStreamOnHGlobal', 563), ('542', 562), ('IsDlgButtonChecked', 557), ('DestroyIcon', 555), ('577', 555), ('__vbaVarVargNofree', 555), ('__vbaGet3', 555), ('IsBadReadPtr', 555), ('GetCurrentDirectoryA', 552), ('23', 546), ('MessageBoxIndirectW', 545), ('EnumSystemLocalesA', 544), ('__vbaVarOr', 539), ('SystemTimeToFileTime', 535), ('LookupPrivilegeValueA', 535), ('__vbaStrFixstr', 533), ('IsIconic', 531), ('GetDriveTypeA', 530), ('GetTokenInformation', 528), ('MoveWindow', 525), ('713', 525), ('FindFirstFileExA', 522), ('FormatMessageA', 522), ('SHGetMalloc', 521), ('SetMapMode', 521), ('CreatePipe', 521), ('FileTimeToLocalFileTime', 520), ('GetKeyState', 519), ('696', 518), ('667', 517), ('VariantCopy', 517), ('AllocateAndInitializeSid', 517), ('__vbaStrI4', 516), ('atoi', 516), ('303', 515), ('free', 514), ('__vbaPut3', 514), ('RegOpenKeyA', 513), ('3', 508), ('__vbaStrI2', 506), ('__vbaUI1I2', 506), ('FormatMessageW', 505), ('__vbaLateIdCallLd', 504), ('309', 504), ('FreeSid', 501), ('UnregisterClassA', 498), ('GetActiveWindow', 497), ('MapWindowPoints', 497), ('GetUserNameA', 495), ('GetPrivateProfileIntW', 494), ('521', 493), ('569', 493), ('WaitForMultipleObjects', 492), ('HeapSetInformation', 488), ('16', 488), ('malloc', 487), ('PathFileExistsA', 482), ('MoveFileExW', 481), ('CLSIDFromString', 481), ('__vbaAryMove', 478), ('__vbaVarSetVar', 477), ('ClientToScreen', 476), ('IsDBCSLeadByte', 476), ('CreateToolhelp32Snapshot', 474), ('InitCommonControlsEx', 473), ('545', 472), ('CreateEventA', 470), ('GetSubMenu', 469), ('CharUpperW', 469), ('RegisterClassExW', 468), ('LocalFileTimeToFileTime', 464), ('MsgWaitForMultipleObjects', 459), ('CopyRect', 457), ('553', 456), ('DestroyMenu', 455), ('SelectClipRgn', 451), ('EnumSystemLocalesW', 451), ('LookupPrivilegeValueW', 450), ('CreateRectRgn', 449), ('memmove', 448), ('19', 445), ('UnhookWindowsHookEx', 442), ('GetComputerNameA', 442), ('__vbaCopyBytes', 442), ('VariantClear', 439), ('702', 439), ('GetAsyncKeyState', 437), ('CoTaskMemAlloc', 435), ('SHGetFolderPathW', 435), ('DeviceIoControl', 435), ('__vbaVarCmpNe', 433), ('FindWindowA', 432), ('547', 430), ('OpenMutexA', 430), ('EnumWindows', 429), ('IsBadWritePtr', 429), ('GetSysColorBrush', 428), ('EVENT_SINK2_Release', 428), ('EVENT_SINK2_AddRef', 428), ('GetMenu', 427), ('_cexit', 427), ('SetActiveWindow', 426), ('TerminateThread', 426), ('SetCapture', 425), ('CreateMutexW', 424), ('GetWindowTextW', 423), ('GetDateFormatW', 422), ('CreateFileMappingW', 421), ('__vbaVarLateMemCallLdRf', 421), ('LocalReAlloc', 420), ('CloseServiceHandle', 420), ('CallNextHookEx', 415), ('PtInRect', 413), ('WinExec', 413), ('ResumeThread', 412), ('CreateFileMappingA', 410), ('GlobalDeleteAtom', 409), ('TextOutA', 409), ('12', 406), ('__vbaVarForInit', 405), ('__vbaVarForNext', 405), ('116', 404), ('__vbaVar2Vec', 403), ('ReleaseCapture', 401), ('AdjustWindowRectEx', 401), ('CryptReleaseContext', 401), ('__vbaVarNot', 401), ('__vbaVarCmpGt', 396), ('OutputDebugStringA', 396), ('GetLastActivePopup', 395), ('_onexit', 395), ('GetMenuItemCount', 394), ('RegisterWindowMessageA', 393), ('ReleaseSemaphore', 392), ('10', 391), ('strncpy', 388), ('CoInitializeEx', 387), ('52', 387), ('__vbaLateMemSt', 386), ('SetPropA', 385), ('544', 383), ('InterlockedCompareExchange', 383), ('669', 382), ('__vbaLateIdCall', 381), ('RasEnumEntriesA', 380), ('11', 380), ('SetRect', 377), ('GetDriveTypeW', 376), ('SHGetSpecialFolderPathA', 376), ('MessageBeep', 375), ('Polyline', 375), ('WaitForSingleObjectEx', 375), ('__dllonexit', 375), ('SetConsoleCtrlHandler', 373), ('FindResourceExW', 372), ('GetTimeFormatW', 371), ('18', 371), ('CLSIDFromProgID', 371), ('DragFinish', 369), ('FreeResource', 369), ('__vbaVarMod', 368), ('strcpy', 368), ('strlen', 368), ('RemovePropA', 367), ('SHChangeNotify', 367), ('GetPropA', 366), ('GetCapture', 366), ('OffsetRect', 365), ('CheckMenuItem', 363), ('__vbaI4Str', 363), ('EqualSid', 363), ('GetClassNameA', 363), ('RedrawWindow', 362), ('GetWindowDC', 362), ('517', 359), ('__vbaLateIdSt', 359), ('sprintf', 358), ('SendDlgItemMessageA', 355), ('strncmp', 355), ('__vbaAryCopy', 354), ('GetWindowPlacement', 353), ('FindWindowW', 352), ('GetTextExtentPoint32W', 350), ('21', 350), ('543', 348), ('GetMenuItemID', 347), ('DosDateTimeToFileTime', 346), ('NtUnmapViewOfSection', 346), ('523', 345), ('GetFileTime', 343), ('CreatePen', 343), ('strcat', 343), ('7', 343), ('__vbaErase', 339), ('DrawFocusRect', 338), ('VariantChangeTypeEx', 338), ('PeekNamedPipe', 334), ('SendDlgItemMessageW', 333), ('DrawIcon', 333), ('__CxxFrameHandler', 333), ('GlobalReAlloc', 330), ('GlobalHandle', 330), ('GetTimeFormatA', 329), ('_stricmp', 328), ('ShellExecuteExW', 327), ('__vbaVarXor', 325), ('CreateMailslotA', 324), ('CreateSemaphoreA', 324), ('PStoreCreateInstance', 323), ('__vbaForEachCollAd', 322), ('__vbaNextEachCollAd', 322), ('ClosePrinter', 322), ('FindFirstFileExW', 321), ('__vbaCastObj', 321), ('SetViewportExtEx', 320), ('20', 319), ('SetWindowsHookExA', 319), ('IsBadCodePtr', 319), ('IsChild', 318), ('GetClassNameW', 318), ('SetWindowExtEx', 318), ('RegQueryInfoKeyA', 315), ('IsDialogMessageA', 315), ('IsValidSid', 314), ('GlobalAddAtomA', 314), ('GetMenuState', 313), ('__vbaR8Str', 312), ('GetEnvironmentVariableW', 312), ('PathAppendW', 311), ('GetMailslotInfo', 310), ('void __cdecl operator delete(void *)', 308), ('GlobalFlags', 307), ('GetDlgCtrlID', 307), ('ScaleWindowExtEx', 306), ('CommandLineToArgvW', 305), ('VariantCopyInd', 304), ('ExtTextOutW', 301), ('GetThreadContext', 300), ('GetOpenFileNameA', 300), ('SysStringLen', 299), ('CharUpperA', 297), ('CryptUnprotectData', 297), ('ScaleViewportExtEx', 296), ('__vbaNextEachAry', 295), ('__vbaForEachAry', 295), ('SetFileSecurityW', 293), ('IntersectRect', 292), ('RegEnumKeyExW', 292), ('GetTopWindow', 291), ('OpenEventW', 291), ('IsZoomed', 290), ('GetVolumeInformationA', 289), ('RegisterWindowMessageW', 288), ('__vbaAryVar', 286), ('GlobalMemoryStatus', 285), ('WinHelpA', 285), ('SetConsoleMode', 284), ('wvsprintfA', 284), ('ProgIDFromCLSID', 283), ('GetWindowTextLengthA', 283), ('RegCreateKeyA', 283), ('GetTextExtentPointA', 282), ('IsRectEmpty', 281), ('GetAtomNameA', 281), ('GetExitCodeThread', 281), ('IsDialogMessageW', 280), ('__vbaI4ErrVar', 280), ('SetMenu', 279), ('GetMessageTime', 279), ('SHGetDesktopFolder', 279), ('QueryPerformanceFrequency', 279), ('void * __cdecl operator new(unsigned int)', 279), ('__vbaFPInt', 278), ('Ellipse', 275), ('GetDateFormatA', 274), ('VirtualAllocEx', 274), ('strstr', 274), ('strcmp', 273), ('DeleteMenu', 269), ('_strnicmp', 269), ('FindAtomA', 268), ('AddAtomA', 267), ('609', 265), ('EndPage', 265), ('Escape', 264)]

func_name_o_t = func_name_o[:50]
func_name_x_t = func_name_x[:50]
print(func_name_o_t)
print(func_name_x_t)

[('GetProcAddress', 4906), ('GetLastError', 4832), ('GetCurrentProcess', 4481), ('CloseHandle', 4473), ('Sleep', 4362), ('MultiByteToWideChar', 4234), ('GetTickCount', 4104), ('ExitProcess', 4001), ('WriteFile', 3895), ('GetCurrentThreadId', 3791), ('GetModuleHandleA', 3647), ('RegCloseKey', 3631), ('FreeLibrary', 3542), ('UnhandledExceptionFilter', 3435), ('QueryPerformanceCounter', 3434), ('GetCurrentProcessId', 3415), ('WideCharToMultiByte', 3375), ('LoadLibraryA', 3265), ('GetSystemTimeAsFileTime', 3246), ('ReadFile', 3228), ('TerminateProcess', 3227), ('SetFilePointer', 3140), ('SetUnhandledExceptionFilter', 3074), ('DeleteCriticalSection', 3067), ('LeaveCriticalSection', 3012), ('EnterCriticalSection', 3009), ('WaitForSingleObject', 2834), ('GetCommandLineA', 2720), ('GetModuleFileNameA', 2610), ('FindClose', 2559), ('GetFileSize', 2545), ('CreateFileA', 2473), ('RtlUnwind', 2447), ('InitializeCriticalSection', 2413), ('LocalFree', 2399), ('_initterm', 2394), ('GetDC', 2394), ('G

In [ ]:
# 정상
list1 = func_name_o_t
# 악성
list2 = func_name_x_t
# 정상에서 뺀 거
dic_o = {}
dic_x = {}
for i in range(len(list1)):
  for j in range(len(list2)):
    if list1[i][0] == list2[j][0]:
      break
  if j == len(list2)-1:
    dic_o[list1[i][0]] = list1[i][1]
print(dic_o)

# 악성에서 뺀 거
for i in range(len(list2)):
  for j in range(len(list1)):
    if list2[i][0] == list1[j][0]:
      break
  if j == len(list1)-1:
    dic_x[list2[i][0]] = list2[i][1]
print(dic_x)

print(len(dic_o))
print(len(dic_x))

{'QueryPerformanceCounter': 3434, 'GetCurrentProcessId': 3415, 'GetSystemTimeAsFileTime': 3246, 'TerminateProcess': 3227, 'SetUnhandledExceptionFilter': 3074, 'InitializeCriticalSection': 2413, 'LocalFree': 2399, '_initterm': 2394, 'GetDC': 2394, 'CoTaskMemFree': 2325, 'CoCreateInstance': 2308, 'free': 2281, 'GetWindowRect': 2262, 'TlsGetValue': 2260}
{'': 159559, 'RegQueryValueExA': 5448, 'RegOpenKeyExA': 5358, 'SelectObject': 5280, 'GetDeviceCaps': 5019, 'SetTextColor': 4967, 'CharNextA': 4890, 'SetBkColor': 4879, 'FindFirstFileA': 4766, 'SetBkMode': 4735, 'DestroyWindow': 4552, 'GetStartupInfoA': 4496, 'GetFileType': 4437, 'lstrcpynA': 4427, 'ImageList_Create': 4411}
14
15
15


In [ ]:
print(dic_o)
print(dic_x)
print(len(dic_o))
print(len(dic_x))

#dic_x = {'VerQueryValueA': 3826, 'GetFileVersionInfoA': 3381, 'GetFileVersionInfoSizeA': 3376, 'lstrcmpiA': 3359, 'HeapReAlloc': 3285, 'FindNextFileA': 3272, 'SetErrorMode': 3241, 'DispatchMessageA': 3241, 'IsWindowEnabled': 3200}